# Read me
## Companion animals, mental health, and well-being during the COVID-19 pandemic

This document contains the original surveys, in French and in English, used
for this study.

The English version has the variable codes used for the original
data extraction.

The dataset includes the 23 variables used in the published manuscript.
* D2: province or territory
* region: Canadian geographical regions divided as British Columbia, Prairies
and Territories (Alberta, Saskatchewan, Manitoba, Nunavut, Yukon, and Northwest
Territories), Ontario, Québec, and Atlantic (New Brunswick, Nova Scotia,
Prince-Edward-Island, and Newfoundland and Labrador)
* O1: pet ownership
* O8b: change in pet ownership
* P1: pet attitude
* D1: age
* D5: highest level of education
* D67: ethnicity
* D4: gender
* D11: number of people in the household
* D8: yearly income
* E3: change in income since the beginning of the pandemic
* H2: disability
* H1Ax: emotional, psychological or mental health condition
* H4: change in mental health since the beginning of the pandemic
* H56: tested positive to COVID-19 (or household member)
* S: social support
* QOL: quality of life (EQ-5D-5L score converted on the scale developed for Canada)
* GH: self-assessed overall health
* L1: loneliness
* H3: perceived mental health
* Q2: self-reported level of stress
* Q1_cat: anxiety

The variables O8b, D67, H56, S, QOL, and Q1_cat were obtained from the
combination of other variables, which can be obtained by contacting the authors.

The R code includes details for descriptive statistics, Bayesian mixed
univariable and multivariable ordinal and linear models, diagnostics, and plots.

Tables with the estimates of models from stratified data (dog owners-only and
cat owners-only) were added on Jan. 31, 2022.


In [3]:
import pandas as pd

data = pd.read_csv("dataset_petandcovid.csv", encoding='ISO-8859-1')


ATE and Estimations

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(123)

# Step 1: Data Preprocessing
def preprocess_data(data, o8b_treated='New', o8b_control='No'):
    print("Starting preprocessing...")
    df = data.copy()

    # Print initial shape and O8b values
    print(f"Initial shape: {df.shape}")
    print("O8b value counts:")
    print(df['O8b'].value_counts(dropna=False))

    # Print available columns
    print("Available columns:")
    print(df.columns.tolist())

    # Define treatment (T) from O8b
    df['treatment'] = np.where(df['O8b'] == o8b_treated, 1,
                             np.where(df['O8b'] == o8b_control, 0, np.nan))

    # Check treatment creation
    print("Treatment value counts:")
    print(df['treatment'].value_counts(dropna=False))

    # Filter to keep only T = 0 or 1
    df = df[df['treatment'].notna()]
    print(f"Shape after filtering treatment: {df.shape}")

    if df.empty:
        raise ValueError("No rows remain after filtering O8b. Check o8b_treated and o8b_control values.")

    # Define outcome (Y)
    outcome = 'QOL'
    if outcome not in df.columns:
        raise ValueError(f"Outcome column '{outcome}' not found in data.")

    # Define confounders (X)
    confounders = ['D4', 'D5', 'D67', 'D11', 'H2', 'H1Ax', 'H56', 'P1', 'region']
    # Keep only available confounders
    confounders = [col for col in confounders if col in df.columns]
    print(f"Available confounders: {confounders}")
    if not confounders:
        print("Warning: No confounders found. Proceeding with treatment and outcome only.")

    # Print missing values
    print("Missing values before imputation:")
    print(df[[outcome, 'treatment'] + confounders].isna().sum())

    # Print sample data for categorical confounders
    print("Sample data for categorical confounders:")
    categorical_sample = [c for c in ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56'] if c in df.columns]
    if categorical_sample:
        print(df[categorical_sample].head())

    # Handle categorical variables with one-hot encoding
    categorical_vars = ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
    categorical_vars = [var for var in categorical_vars if var in confounders]
    print(f"One-hot encoding categorical variables: {categorical_vars}")
    if categorical_vars:
        df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)  # Use int dtype

    # Update confounder list to include dummy variables
    confounder_cols = [col for col in df.columns if any(col.startswith(var + '_') for var in categorical_vars)]
    confounder_cols += [var for var in confounders if var not in categorical_vars]
    print(f"Confounder columns after encoding: {confounder_cols}")

    # Print dtypes of dummy variables
    print("Dtypes of dummy variables:")
    dummy_cols = [col for col in confounder_cols if any(col.startswith(var + '_') for var in categorical_vars)]
    if dummy_cols:
        print(df[dummy_cols].dtypes)

    # Define continuous variables
    continuous_vars = ['D11', 'P1']
    continuous_vars = [var for var in continuous_vars if var in df.columns]
    print(f"Continuous variables: {continuous_vars}")

    # Convert continuous variables to numeric
    for var in continuous_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    # Check for valid continuous variables (non-empty after conversion)
    valid_continuous_vars = [var for var in continuous_vars if df[var].notna().any()]
    print(f"Valid continuous variables (non-empty): {valid_continuous_vars}")

    # Impute and standardize continuous variables
    if valid_continuous_vars:
        print(f"Imputing and standardizing: {valid_continuous_vars}")
        imputer = SimpleImputer(strategy='mean')
        imputed_data = imputer.fit_transform(df[valid_continuous_vars])
        if imputed_data.shape[1] != len(valid_continuous_vars):
            raise ValueError(f"Imputation output has {imputed_data.shape[1]} columns, expected {len(valid_continuous_vars)}")
        df[valid_continuous_vars] = imputed_data

        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(df[valid_continuous_vars])
        df[valid_continuous_vars] = scaled_data
    else:
        print("No valid continuous variables for standardization.")

    # Impute categorical confounders only if missing values exist
    categorical_confounders = [col for col in confounder_cols if col not in continuous_vars]
    if categorical_confounders:
        missing_cat = df[categorical_confounders].isna().any().any()
        print(f"Missing values in categorical confounders: {missing_cat}")
        if missing_cat:
            print(f"Imputing categorical confounders: {categorical_confounders}")
            imputer_cat = SimpleImputer(strategy='most_frequent')
            imputed_cat_data = imputer_cat.fit_transform(df[categorical_confounders])
            if imputed_cat_data.shape[1] != len(categorical_confounders):
                raise ValueError(f"Categorical imputation output has {imputed_cat_data.shape[1]} columns, expected {len(categorical_confounders)}")
            df[categorical_confounders] = imputed_cat_data
        else:
            print("No imputation needed for categorical confounders (no missing values).")

    # Impute outcome if missing
    if df[outcome].isna().any():
        print("Imputing missing QOL values...")
        imputer_outcome = SimpleImputer(strategy='mean')
        df[outcome] = imputer_outcome.fit_transform(df[[outcome]])

    # Check data types of confounders
    print("Data types of confounder columns:")
    print(df[confounder_cols].dtypes)

    # Ensure all confounders are numeric
    non_numeric_cols = df[confounder_cols].select_dtypes(exclude=['int64', 'float64', 'uint8', 'int32']).columns
    if non_numeric_cols.any():
        raise ValueError(f"Non-numeric confounders detected: {non_numeric_cols.tolist()}")

    # Final check for missing values
    print("Missing values after imputation:")
    final_cols = [outcome, 'treatment'] + (confounder_cols if confounder_cols else [])
    print(df[final_cols].isna().sum())

    print(f"Final shape after preprocessing: {df.shape}")

    if df.empty:
        raise ValueError("No data remains after preprocessing.")

    return df, outcome, confounder_cols

# Step 2: AIPTW Estimator
def estimate_aiptw(df, outcome, confounders):
    T = df['treatment'].values
    Y = df[outcome].values
    X = df[confounders].values if confounders else np.ones((len(df), 1))  # Fallback for no confounders

    ps_model = LogisticRegression(max_iter=1000).fit(X, T)
    ps = ps_model.predict_proba(X)[:, 1]

    treated_idx = T == 1
    control_idx = T == 0

    outcome_model_treated = LinearRegression().fit(X[treated_idx], Y[treated_idx])
    outcome_model_control = LinearRegression().fit(X[control_idx], Y[control_idx])

    mu1_hat = outcome_model_treated.predict(X)
    mu0_hat = outcome_model_control.predict(X)

    term1 = (T * Y / ps) + (1 - T / ps) * mu1_hat
    term0 = ((1 - T) * Y / (1 - ps)) + (1 - (1 - T) / (1 - ps)) * mu0_hat

    mu1 = np.mean(term1)
    mu0 = np.mean(term0)
    ate = mu1 - mu0

    n = len(Y)
    boot_ates = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        T_boot = T[idx]
        Y_boot = Y[idx]
        X_boot = X[idx]
        ps_boot = ps_model.predict_proba(X_boot)[:, 1]
        mu1_boot = outcome_model_treated.predict(X_boot)
        mu0_boot = outcome_model_control.predict(X_boot)
        term1_boot = (T_boot * Y_boot / ps_boot) + (1 - T_boot / ps_boot) * mu1_boot
        term0_boot = ((1 - T_boot) * Y_boot / (1 - ps_boot)) + (1 - (1 - T_boot) / (1 - ps_boot)) * mu0_boot
        boot_ates.append(np.mean(term1_boot) - np.mean(term0_boot))

    se = np.std(boot_ates)
    ci = (ate - 1.96 * se, ate + 1.96 * se)

    return ate, se, ci, ps

# Step 3: IPTW Estimator
def estimate_iptw(df, outcome, confounders):
    T = df['treatment'].values
    Y = df[outcome].values
    X = df[confounders].values if confounders else np.ones((len(df), 1))

    ps_model = LogisticRegression(max_iter=1000).fit(X, T)
    ps = ps_model.predict_proba(X)[:, 1]

    weights = T / ps + (1 - T) / (1 - ps)
    weights = np.clip(weights, 0.1, 10)

    mu1 = np.sum((T * Y * weights)) / np.sum(T * weights)
    mu0 = np.sum(((1 - T) * Y * weights)) / np.sum((1 - T) * weights)
    ate = mu1 - mu0

    df['weight'] = weights
    model = sm.WLS(Y, sm.add_constant(T), weights=weights).fit(cov_type='HC3')
    se = model.bse[1]
    ci = (ate - 1.96 * se, ate + 1.96 * se)

    return ate, se, ci, ps

# Step 4: Regression Adjustment
def estimate_regression(df, outcome, confounders):
    X = df[['treatment'] + confounders].copy() if confounders else df[['treatment']].copy()
    X = sm.add_constant(X)
    Y = df[outcome]

    model = sm.OLS(Y, X).fit(cov_type='HC3')
    ate = model.params['treatment']
    se = model.bse['treatment']
    ci = (ate - 1.96 * se, ate + 1.96 * se)

    return ate, se, ci

# Step 5: Diagnostics
def plot_diagnostics(ps, df, output_dir='plots'):
    os.makedirs(output_dir, exist_ok=True)

    plt.figure(figsize=(8, 6))
    sns.histplot(ps[df['treatment'] == 1], label='Treated', color='blue', alpha=0.5)
    sns.histplot(ps[df['treatment'] == 0], label='Control', color='orange', alpha=0.5)
    plt.title('Propensity Score Distribution')
    plt.xlabel('Propensity Score')
    plt.legend()
    plt.savefig(f'{output_dir}/ps_overlap.png')
    plt.close()

    plt.figure(figsize=(8, 6))
    sns.histplot(df['weight'], color='green', alpha=0.5)
    plt.title('IPTW Weight Distribution')
    plt.xlabel('Weight')
    plt.savefig(f'{output_dir}/weight_distribution.png')
    plt.close()

# Main function
def main(data, o8b_treated='New', o8b_control='No'):
    df, outcome, confounders = preprocess_data(data, o8b_treated, o8b_control)

    print("Estimating ATE...")

    ate_aiptw, se_aiptw, ci_aiptw, ps = estimate_aiptw(df, outcome, confounders)
    print(f"AIPTW: ATE = {ate_aiptw:.3f}, SE = {se_aiptw:.3f}, 95% CI = ({ci_aiptw[0]:.3f}, {ci_aiptw[1]:.3f})")

    ate_iptw, se_iptw, ci_iptw, _ = estimate_iptw(df, outcome, confounders)
    print(f"IPTW: ATE = {ate_iptw:.3f}, SE = {se_iptw:.3f}, 95% CI = ({ci_iptw[0]:.3f}, {ci_iptw[1]:.3f})")

    ate_reg, se_reg, ci_reg = estimate_regression(df, outcome, confounders)
    print(f"Regression: ATE = {ate_reg:.3f}, SE = {se_reg:.3f}, 95% CI = ({ci_reg[0]:.3f}, {ci_reg[1]:.3f})")

    plot_diagnostics(ps, df)
    print("Diagnostic plots saved in 'plots' directory.")

# Run the analysis
if __name__ == "__main__":
    main(data, o8b_treated='New', o8b_control='No')

Starting preprocessing...
Initial shape: (1500, 23)
O8b value counts:
O8b
No      1259
Loss     173
New       68
Name: count, dtype: int64
Available columns:
['D2', 'region', 'O1', 'O8b', 'P1', 'D1', 'D5', 'D67', 'D4', 'D11', 'D8', 'E3', 'H2', 'H1Ax', 'H4', 'H56', 'S', 'QOL', 'GH', 'L1', 'H3', 'Q2', 'Q1_cat']
Treatment value counts:
treatment
0.0    1259
NaN     173
1.0      68
Name: count, dtype: int64
Shape after filtering treatment: (1327, 24)
Available confounders: ['D4', 'D5', 'D67', 'D11', 'H2', 'H1Ax', 'H56', 'P1', 'region']
Missing values before imputation:
QOL          0
treatment    0
D4           0
D5           0
D67          0
D11          0
H2           0
H1Ax         0
H56          0
P1           0
region       0
dtype: int64
Sample data for categorical confounders:
      D4           D5         D67    region   H2 H1Ax H56
0    Men  High school      Others  Atlantic   No   No  No
1    Men      College  Caucasians  Prairies   No   No  No
2    Men   University  Caucasians  

Matching, ATT, and Sensitivity Analysis

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(123)

# Step 1: Data Preprocessing
def preprocess_data(data, o8b_treated='New', o8b_control='No'):
    print("Starting preprocessing...")
    df = data.copy()

    print(f"Initial shape: {df.shape}")
    print("O8b value counts:")
    print(df['O8b'].value_counts(dropna=False))

    print("Available columns:")
    print(df.columns.tolist())

    df['treatment'] = np.where(df['O8b'] == o8b_treated, 1,
                             np.where(df['O8b'] == o8b_control, 0, np.nan))

    print("Treatment value counts:")
    print(df['treatment'].value_counts(dropna=False))

    df = df[df['treatment'].notna()]
    df = df.reset_index(drop=True)
    print(f"Shape after filtering treatment: {df.shape}")

    if df.empty:
        raise ValueError("No rows remain after filtering O8b. Check o8b_treated and o8b_control values.")

    outcome = 'QOL'
    if outcome not in df.columns:
        raise ValueError(f"Outcome column '{outcome}' not found in data.")

    confounders = ['D4', 'D5', 'D67', 'D11', 'H2', 'H1Ax', 'H56', 'P1', 'region']
    confounders = [col for col in confounders if col in df.columns]
    print(f"Available confounders: {confounders}")

    print("Missing values before imputation:")
    print(df[[outcome, 'treatment'] + confounders].isna().sum())

    print("Sample data for categorical confounders:")
    categorical_sample = [c for c in ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56'] if c in df.columns]
    if categorical_sample:
        print(df[categorical_sample].head())

    categorical_vars = ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
    categorical_vars = [var for var in categorical_vars if var in confounders]
    print(f"One-hot encoding categorical variables: {categorical_vars}")
    if categorical_vars:
        df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)

    confounder_cols = [col for col in df.columns if any(col.startswith(var + '_') for var in categorical_vars)]
    confounder_cols += [var for var in confounders if var not in categorical_vars]
    print(f"Confounder columns after encoding: {confounder_cols}")

    continuous_vars = ['D11', 'P1']
    continuous_vars = [var for var in continuous_vars if var in df.columns]
    print(f"Continuous variables: {continuous_vars}")

    for var in continuous_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    valid_continuous_vars = [var for var in continuous_vars if df[var].notna().any()]
    print(f"-utils continuous variables (non-empty): {valid_continuous_vars}")

    if valid_continuous_vars:
        print(f"Imputing and standardizing: {valid_continuous_vars}")
        imputer = SimpleImputer(strategy='mean')
        df[valid_continuous_vars] = imputer.fit_transform(df[valid_continuous_vars])

        scaler = StandardScaler()
        df[valid_continuous_vars] = scaler.fit_transform(df[valid_continuous_vars])

    categorical_confounders = [col for col in confounder_cols if col not in continuous_vars]
    if categorical_confounders:
        missing_cat = df[categorical_confounders].isna().any().any()
        print(f"Missing values in categorical confounders: {missing_cat}")
        if missing_cat:
            print(f"Imputing categorical confounders: {categorical_confounders}")
            imputer_cat = SimpleImputer(strategy='most_frequent')
            df[categorical_confounders] = imputer_cat.fit_transform(df[categorical_confounders])

    if df[outcome].isna().any():
        print("Imputing missing QOL values...")
        imputer_outcome = SimpleImputer(strategy='mean')
        df[outcome] = imputer_outcome.fit_transform(df[[outcome]])

    print("Data types of confounder columns:")
    print(df[confounder_cols].dtypes)

    non_numeric_cols = df[confounder_cols].select_dtypes(exclude=['int64', 'float64', 'uint8', 'int32']).columns
    if non_numeric_cols.any():
        raise ValueError(f"Non-numeric confounders detected: {non_numeric_cols.tolist()}")

    print("Missing values after imputation:")
    final_cols = [outcome, 'treatment'] + (confounder_cols if confounder_cols else [])
    print(df[final_cols].isna().sum())

    print(f"Final shape after preprocessing: {df.shape}")

    if df.empty:
        raise ValueError("No data remains after preprocessing.")

    return df, outcome, confounder_cols

# Step 2: Propensity Score Matching
def propensity_score_matching(df, confounders):
    ps_model = LogisticRegression(max_iter=1000).fit(df[confounders], df['treatment'])
    ps = ps_model.predict_proba(df[confounders])[:, 1]

    treated_idx = df[df['treatment'] == 1].index
    control_idx = df[df['treatment'] == 0].index

    treated_ps = ps[treated_idx]
    control_ps = ps[control_idx]

    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control_ps.reshape(-1, 1))
    distances, indices = nn.kneighbors(treated_ps.reshape(-1, 1))

    matched_control_idx = control_idx[indices.flatten()]
    matched_df = df.loc[list(treated_idx) + list(matched_control_idx)]

    print(f"Shape of matched dataset: {matched_df.shape}")
    return matched_df, ps

# Step 3: IPTW for ATT
def estimate_iptw_att(df, outcome, confounders):
    T = df['treatment'].values
    Y = df[outcome].values
    X = df[confounders].values if confounders else np.ones((len(df), 1))

    ps_model = LogisticRegression(max_iter=1000).fit(X, T)
    ps = ps_model.predict_proba(X)[:, 1]

    weights = np.where(T == 1, 1, ps / (1 - ps))
    weights = np.clip(weights, 0.1, 10)

    treated_idx = T == 1
    control_idx = T == 0

    mu1 = np.mean(Y[treated_idx])
    mu0 = np.average(Y[control_idx], weights=weights[control_idx])
    att = mu1 - mu0

    n = len(Y)
    boot_atts = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        T_boot = T[idx]
        Y_boot = Y[idx]
        weights_boot = weights[idx]
        treated_boot = T_boot == 1
        control_boot = T_boot == 0
        mu1_boot = np.mean(Y_boot[treated_boot])
        mu0_boot = np.average(Y_boot[control_boot], weights=weights_boot[control_boot])
        boot_atts.append(mu1_boot - mu0_boot)

    se = np.std(boot_atts)
    ci = (att - 1.96 * se, att + 1.96 * se)

    df['weight'] = weights
    return att, se, ci, ps

# Step 4: AIPTW (for Matched Data)
def estimate_aiptw(df, outcome, confounders):
    T = df['treatment'].values
    Y = df[outcome].values
    X = df[confounders].values if confounders else np.ones((len(df), 1))

    ps_model = LogisticRegression(max_iter=1000).fit(X, T)
    ps = ps_model.predict_proba(X)[:, 1]

    treated_idx = T == 1
    control_idx = T == 0

    outcome_model_treated = LinearRegression().fit(X[treated_idx], Y[treated_idx])
    outcome_model_control = LinearRegression().fit(X[control_idx], Y[control_idx])

    mu1_hat = outcome_model_treated.predict(X)
    mu0_hat = outcome_model_control.predict(X)

    term1 = (T * Y / ps) + (1 - T / ps) * mu1_hat
    term0 = ((1 - T) * Y / (1 - ps)) + (1 - (1 - T) / (1 - ps)) * mu0_hat

    mu1 = np.mean(term1)
    mu0 = np.mean(term0)
    ate = mu1 - mu0

    n = len(Y)
    boot_ates = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        T_boot = T[idx]
        Y_boot = Y[idx]
        X_boot = X[idx]
        ps_boot = ps_model.predict_proba(X_boot)[:, 1]
        mu1_boot = outcome_model_treated.predict(X_boot)
        mu0_boot = outcome_model_control.predict(X_boot)
        term1_boot = (T_boot * Y_boot / ps_boot) + (1 - T_boot / ps_boot) * mu1_boot
        term0_boot = ((1 - T_boot) * Y_boot / (1 - ps_boot)) + (1 - (1 - T_boot) / (1 - ps_boot)) * mu0_boot
        boot_ates.append(np.mean(term1_boot) - np.mean(term0_boot))

    se = np.std(boot_ates)
    ci = (ate - 1.96 * se, ate + 1.96 * se)

    return ate, se, ci, ps

# Step 5: Matching for ATT
def matching_att(df, outcome, ps):
    treated_idx = df[df['treatment'] == 1].index
    control_idx = df[df['treatment'] == 0].index

    treated_ps = ps[treated_idx]
    control_ps = ps[control_idx]

    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control_ps.reshape(-1, 1))
    distances, indices = nn.kneighbors(treated_ps.reshape(-1, 1))

    matched_control_idx = control_idx[indices.flatten()]
    matched_df = df.loc[list(treated_idx) + list(matched_control_idx)]

    att = matched_df[matched_df['treatment'] == 1][outcome].mean() - matched_df[matched_df['treatment'] == 0][outcome].mean()

    boot_atts = []
    for _ in range(200):
        idx = np.random.choice(len(matched_df), len(matched_df), replace=True)
        boot_df = matched_df.iloc[idx]
        att_boot = boot_df[boot_df['treatment'] == 1][outcome].mean() - boot_df[boot_df['treatment'] == 0][outcome].mean()
        boot_atts.append(att_boot)

    se = np.std(boot_atts)
    ci = (att - 1.96 * se, att + 1.96 * se)

    return att, se, ci

# Step 6: Stratified Analysis by Pet Type
def stratified_analysis(df, outcome, confounders):
    print("O1 value counts:")
    print(df['O1'].value_counts(dropna=False))

    pet_types = ['Dog', 'Cat']
    results = {}

    for pet_type in pet_types:
        print(f"\nStratified analysis for {pet_type} owners:")
        sub_df = df[df['O1'].str.contains(pet_type, case=False, na=False)]
        if len(sub_df) < 10 or sub_df['treatment'].nunique() < 2:
            print(f"Skipping {pet_type}: insufficient data (n={len(sub_df)})")
            continue

        att, se, ci, ps = estimate_iptw_att(sub_df, outcome, confounders)
        results[pet_type] = {'ATT': att, 'SE': se, 'CI': ci}
        print(f"{pet_type} ATT = {att:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")

    if not results:
        print("\nTrying binary O1 (pet owners vs. non-owners)...")
        pet_owners = df[df['O1'].str.contains('Yes', case=False, na=False)]
        if len(pet_owners) >= 10 and pet_owners['treatment'].nunique() == 2:
            att, se, ci, ps = estimate_iptw_att(pet_owners, outcome, confounders)
            results['Pet Owners'] = {'ATT': att, 'SE': se, 'CI': ci}
            print(f"Pet Owners ATT = {att:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")

    return results

# Step 7: Sensitivity Analysis (E-value)
def compute_e_value(ate, se):
    point_est = abs(ate)
    rr_point = np.exp(0.91 * point_est)
    e_value_point = rr_point + np.sqrt(rr_point * (rr_point - 1))

    lower_bound = abs(ate - 1.96 * se)
    if lower_bound > 0:
        rr_lower = np.exp(0.91 * lower_bound)
        e_value_lower = rr_lower + np.sqrt(rr_lower * (rr_lower - 1))
    else:
        e_value_lower = 1.0

    return e_value_point, e_value_lower

# Step 8: Calculate R^2 for Observed Confounders
def calculate_r2(df, confounder_cols, target):
    r2_values = []
    for col in confounder_cols:
        X = df[[col]].dropna()
        y = df.loc[X.index, target]
        if len(X) == 0 or y.isna().all():
            continue
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        r2 = r2_score(y, y_pred)
        r2_values.append((col, r2))
    return r2_values

# Step 9: Austin Plot with Bias Curve
def austin_plot_sensitivity(df, outcome, confounder_cols, att_orig, se_orig, output_dir='plots', prefix=''):
    print(f"\nGenerating Austin plot for sensitivity analysis ({prefix})...")

    T = df['treatment'].values
    Y = df[outcome].values

    # Compute propensity scores
    ps_model = LogisticRegression(max_iter=1000).fit(df[confounder_cols], T)
    p = ps_model.predict_proba(df[confounder_cols])[:, 1]

    # Generate U ~ N(0,1)
    U = np.random.normal(0, 1, len(df))

    # Define R^2 grid
    r2_values = np.linspace(0.0, 0.5, 21)
    adjusted_atts = np.zeros((len(r2_values), len(r2_values)))

    sd_y = np.std(Y)

    for i, r2_ut in enumerate(r2_values):
        for j, r2_uy in enumerate(r2_values):
            # Simulate U correlated with propensity score
            a = np.sqrt(r2_ut)
            b = np.sqrt(1 - r2_ut)
            p_std = (p - np.mean(p)) / np.std(p)
            U_sim = a * p_std + b * U

            # Adjust outcome for confounding
            gamma = np.sqrt(r2_uy) * sd_y
            Y_adj = Y - gamma * U_sim

            # Estimate ATT on adjusted outcome
            treated_idx = T == 1
            control_idx = T == 0
            weights = df['weight'].values
            mu1 = np.mean(Y_adj[treated_idx])
            mu0 = np.average(Y_adj[control_idx], weights=weights[control_idx])
            adjusted_atts[i, j] = mu1 - mu0

    # Compute R^2 for observed confounders
    r2_t = calculate_r2(df, confounder_cols, 'treatment')
    r2_y = calculate_r2(df, confounder_cols, outcome)
    confounder_r2 = {conf_t: (r2_ut, r2_uy) for (conf_t, r2_ut), (conf_y, r2_uy) in zip(r2_t, r2_y) if conf_t == conf_y}

    # Colors and markers
    colors = list(mcolors.TABLEAU_COLORS.values())[:len(confounder_r2)]
    if len(colors) < len(confounder_r2):
        colors.extend(list(mcolors.CSS4_COLORS.values())[:len(confounder_r2) - len(colors)])
    markers = ['o', 's', '^', 'D', 'v', '<', '>', 'p', 'h', '*']

    # Plotting
    plt.figure(figsize=(12, 10))
    contour = plt.contourf(r2_values, r2_values, adjusted_atts, levels=20, cmap='viridis')
    plt.colorbar(contour, label='Adjusted ATT', pad=0.1)

    # Contour for ATT = 0
    plt.contour(r2_values, r2_values, adjusted_atts, levels=[0], colors='red', linestyles='dashed', linewidths=2)

    # Bias curve (ATT = lower CI)
    ci_lower = att_orig - 1.96 * se_orig
    cs_ci = plt.contour(r2_values, r2_values, adjusted_atts, levels=[ci_lower], colors='cyan', linestyles='solid', linewidths=2)
    plt.clabel(cs_ci, inline=True, fontsize=10, fmt=f'ATT = {ci_lower:.3f} (Lower CI)')

    # Plot observed confounders
    for idx, (conf, (r2_ut, r2_uy)) in enumerate(confounder_r2.items()):
        plt.scatter(r2_ut, r2_uy, label=conf, color=colors[idx % len(colors)],
                    marker=markers[idx % len(markers)], s=150, alpha=0.8, edgecolors='black')

    # Annotations
    plt.text(0.02, 0.48, f'Original ATT: {att_orig:.3f}', color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.5))
    plt.text(0.02, 0.44, f'CI: ({ci_lower:.3f}, {att_orig + 1.96 * se_orig:.3f})', color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.5))

    plt.xlabel('$R^2_{UT}$ (Influence on Treatment)', fontsize=14, fontweight='bold')
    plt.ylabel('$R^2_{UY}$ (Influence on Outcome)', fontsize=14, fontweight='bold')
    plt.title('Austin Plot for Sensitivity to Unmeasured Confounding', fontsize=16, fontweight='bold', pad=20)

    num_confounders = len(confounder_r2)
    num_cols = min(3, max(1, num_confounders // 4 + 1))
    plt.legend(title="Observed Confounders", fontsize=10, title_fontsize=12,
               loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=num_cols)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/austin_plot_{prefix}.png', bbox_inches='tight')
    plt.close()

# Step 10: Diagnostics with SMD
def compute_smd(df, confounders, weights):
    treated = df[df['treatment'] == 1]
    control = df[df['treatment'] == 0]
    weights_t = weights[df['treatment'] == 1]
    weights_c = weights[df['treatment'] == 0]
    smd_results = {}

    for col in confounders:
        mean_t = np.average(treated[col], weights=weights_t)
        mean_c = np.average(control[col], weights=weights_c)
        var_t = np.average((treated[col] - mean_t)**2, weights=weights_t)
        var_c = np.average((control[col] - mean_c)**2, weights=weights_c)
        smd = (mean_t - mean_c) / np.sqrt((var_t + var_c) / 2)
        smd_results[col] = smd

    return smd_results

def plot_diagnostics(ps, df, output_dir='plots', prefix=''):
    os.makedirs(output_dir, exist_ok=True)

    plt.figure(figsize=(8, 6))
    sns.histplot(ps[df['treatment'] == 1], label='Treated', color='blue', alpha=0.5)
    sns.histplot(ps[df['treatment'] == 0], label='Control', color='orange', alpha=0.5)
    plt.title(f'Propensity Score Distribution ({prefix})')
    plt.xlabel('Propensity Score')
    plt.legend()
    plt.savefig(f'{output_dir}/{prefix}_ps_overlap.png')
    plt.close()

    plt.figure(figsize=(8, 6))
    sns.histplot(df['weight'], color='green', alpha=0.5)
    plt.title(f'IPTW Weight Distribution ({prefix})')
    plt.xlabel('Weight')
    plt.savefig(f'{output_dir}/{prefix}_weight_distribution.png')
    plt.close()

# Main Function
def main(data, o8b_treated='New', o8b_control='No'):
    df, outcome, confounders = preprocess_data(data, o8b_treated, o8b_control)

    print("\nEstimating ATT with IPTW...")
    att_iptw, se_iptw, ci_iptw, ps = estimate_iptw_att(df, outcome, confounders)
    print(f"IPTW ATT = {att_iptw:.3f}, SE = {se_iptw:.3f}, 95% CI = ({ci_iptw[0]:.3f}, {ci_iptw[1]:.3f})")

    print("\nPerforming propensity score matching...")
    matched_df, ps_matched = propensity_score_matching(df, confounders)
    ate_matched, se_matched, ci_matched, ps_matched_new = estimate_aiptw(matched_df, outcome, confounders)
    print(f"Matched ATE (AIPTW) = {ate_matched:.3f}, SE = {se_matched:.3f}, 95% CI = ({ci_matched[0]:.3f}, {ci_matched[1]:.3f})")

    att_matched, se_matched_att, ci_matched_att = matching_att(df, outcome, ps)
    print(f"Matched ATT = {att_matched:.3f}, SE = {se_matched_att:.3f}, 95% CI = ({ci_matched_att[0]:.3f}, {ci_matched_att[1]:.3f})")

    print("\nPerforming stratified analysis...")
    stratified_results = stratified_analysis(df, outcome, confounders)

    print("\nComputing E-value for IPTW ATT...")
    e_value_point, e_value_lower = compute_e_value(att_iptw, se_iptw)
    print(f"E-value (point estimate) = {e_value_point:.2f}")
    print(f"E-value (lower CI bound) = {e_value_lower:.2f}")

    # Add Austin plot
    austin_plot_sensitivity(df, outcome, confounders, att_iptw, se_iptw, prefix='iptw_att')

    print("\nComputing SMD for IPTW ATT...")
    smd_results = compute_smd(df, confounders, df['weight'])
    for col, smd in smd_results.items():
        print(f"SMD for {col}: {smd:.3f}")

    plot_diagnostics(ps, df, prefix='full')
    plot_diagnostics(ps_matched_new, matched_df, prefix='matched')
    print("Diagnostic plots saved in 'plots' directory.")

if __name__ == "__main__":
    main(data, o8b_treated='New', o8b_control='No')

Starting preprocessing...
Initial shape: (1500, 23)
O8b value counts:
O8b
No      1259
Loss     173
New       68
Name: count, dtype: int64
Available columns:
['D2', 'region', 'O1', 'O8b', 'P1', 'D1', 'D5', 'D67', 'D4', 'D11', 'D8', 'E3', 'H2', 'H1Ax', 'H4', 'H56', 'S', 'QOL', 'GH', 'L1', 'H3', 'Q2', 'Q1_cat']
Treatment value counts:
treatment
0.0    1259
NaN     173
1.0      68
Name: count, dtype: int64
Shape after filtering treatment: (1327, 24)
Available confounders: ['D4', 'D5', 'D67', 'D11', 'H2', 'H1Ax', 'H56', 'P1', 'region']
Missing values before imputation:
QOL          0
treatment    0
D4           0
D5           0
D67          0
D11          0
H2           0
H1Ax         0
H56          0
P1           0
region       0
dtype: int64
Sample data for categorical confounders:
      D4           D5         D67    region   H2 H1Ax H56
0    Men  High school      Others  Atlantic   No   No  No
1    Men      College  Caucasians  Prairies   No   No  No
2    Men   University  Caucasians  

<ipython-input-12-cd0bcf1ffdd0>:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = weights



Computing SMD for IPTW ATT...
SMD for D4_Other or did not answer: 0.123
SMD for D4_Women: 0.090
SMD for D5_College: 0.117
SMD for D5_High school: 0.125
SMD for D5_University: -0.207
SMD for D67_Others: -0.195
SMD for region_British Columbia: -0.295
SMD for region_Ontario: -0.074
SMD for region_Prairies: 0.089
SMD for region_Québec: 0.186
SMD for H2_Yes: -0.073
SMD for H1Ax_Yes: 0.139
SMD for H56_Yes: 0.136
SMD for D11: 0.198
SMD for P1: 0.507
Diagnostic plots saved in 'plots' directory.


In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
import os

# Set random seed for reproducibility
np.random.seed(123)

# Step 1: Data Preprocessing
def preprocess_data(data, o8b_treated='New', o8b_control='No'):
    print("Starting preprocessing...")
    df = data.copy()

    print(f"Initial shape: {df.shape}")
    print("O8b value counts:")
    print(df['O8b'].value_counts(dropna=False))

    print("Available columns:")
    print(df.columns.tolist())

    df['treatment'] = np.where(df['O8b'] == o8b_treated, 1,
                             np.where(df['O8b'] == o8b_control, 0, np.nan))

    print("Treatment value counts:")
    print(df['treatment'].value_counts(dropna=False))

    df = df[df['treatment'].notna()]
    df = df.reset_index(drop=True)
    print(f"Shape after filtering treatment: {df.shape}")

    if df.empty:
        raise ValueError("No rows remain after filtering O8b. Check o8b_treated and o8b_control values.")

    outcome = 'QOL'
    if outcome not in df.columns:
        raise ValueError(f"Outcome column '{outcome}' not found in data.")

    confounders = ['D4', 'D5', 'D67', 'D11', 'H2', 'H1Ax', 'H56', 'P1', 'region']
    confounders = [col for col in confounders if col in df.columns]
    print(f"Available confounders: {confounders}")

    print("Missing values before imputation:")
    print(df[[outcome, 'treatment'] + confounders].isna().sum())

    print("Sample data for categorical confounders:")
    categorical_sample = [c for c in ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56'] if c in df.columns]
    if categorical_sample:
        print(df[categorical_sample].head())

    categorical_vars = ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
    categorical_vars = [var for var in categorical_vars if var in confounders]
    print(f"One-hot encoding categorical variables: {categorical_vars}")
    if categorical_vars:
        df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)

    confounder_cols = [col for col in df.columns if any(col.startswith(var + '_') for var in categorical_vars)]
    confounder_cols += [var for var in confounders if var not in categorical_vars]
    print(f"Confounder columns after encoding: {confounder_cols}")

    continuous_vars = ['D11', 'P1']
    continuous_vars = [var for var in continuous_vars if var in df.columns]
    print(f"Continuous variables: {continuous_vars}")

    for var in continuous_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    valid_continuous_vars = [var for var in continuous_vars if var in df.columns and df[var].notna().any()]
    print(f"Valid continuous variables (non-empty): {valid_continuous_vars}")

    if valid_continuous_vars:
        print(f"Imputing and standardizing: {valid_continuous_vars}")
        imputer = SimpleImputer(strategy='mean')
        df[valid_continuous_vars] = imputer.fit_transform(df[valid_continuous_vars])

        scaler = StandardScaler()
        df[valid_continuous_vars] = scaler.fit_transform(df[valid_continuous_vars])

    categorical_confounders = [col for col in confounder_cols if col not in continuous_vars]
    if categorical_confounders:
        missing_cat = df[categorical_confounders].isna().any().any()
        print(f"Missing values in categorical confounders: {missing_cat}")
        if missing_cat:
            print(f"Imputing categorical confounders: {categorical_confounders}")
            imputer_cat = SimpleImputer(strategy='most_frequent')
            df[categorical_confounders] = imputer_cat.fit_transform(df[categorical_confounders])

    if df[outcome].isna().any():
        print("Imputing missing QOL values...")
        imputer_outcome = SimpleImputer(strategy='mean')
        df[outcome] = imputer_outcome.fit_transform(df[[outcome]])

    print("Data types of confounder columns:")
    print(df[confounder_cols].dtypes)

    non_numeric_cols = df[confounder_cols].select_dtypes(exclude=['int64', 'float64', 'uint8', 'int32']).columns
    if non_numeric_cols.any():
        raise ValueError(f"Non-numeric confounders detected: {non_numeric_cols.tolist()}")

    print("Missing values after imputation:")
    final_cols = [outcome, 'treatment'] + (confounder_cols if confounder_cols else [])
    print(df[final_cols].isna().sum())

    print(f"Final shape after preprocessing: {df.shape}")

    if df.empty:
        raise ValueError("No data remains after preprocessing.")

    return df, outcome, confounder_cols

# Step 2: Propensity Score Matching (Fixed)
def propensity_score_matching(df, confounders):
    ps_model = LogisticRegression(max_iter=1000).fit(df[confounders], df['treatment'])
    ps = ps_model.predict_proba(df[confounders])[:, 1]

    treated_idx = df[df['treatment'] == 1].index
    control_idx = df[df['treatment'] == 0].index

    treated_ps = ps[treated_idx]
    control_ps = ps[control_idx]

    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control_ps.reshape(-1, 1))
    distances, indices = nn.kneighbors(treated_ps.reshape(-1, 1))

    matched_control_idx = control_idx[indices.flatten()]
    matched_df = df.loc[list(treated_idx) + list(matched_control_idx)]

    print(f"Shape of matched dataset: {matched_df.shape}")
    return matched_df, ps

# Step 3: IPTW for ATT
def estimate_iptw_att(df, outcome, confounders):
    T = df['treatment'].values
    Y = df[outcome].values
    X = df[confounders].values if confounders else np.ones((len(df), 1))

    ps_model = LogisticRegression(max_iter=1000).fit(X, T)
    ps = ps_model.predict_proba(X)[:, 1]

    weights = np.where(T == 1, 1, ps / (1 - ps))
    weights = np.clip(weights, 0.1, 10)

    treated_idx = T == 1
    control_idx = T == 0

    mu1 = np.mean(Y[treated_idx])
    mu0 = np.average(Y[control_idx], weights=weights[control_idx])
    att = mu1 - mu0

    n = len(Y)
    boot_atts = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        T_boot = T[idx]
        Y_boot = Y[idx]
        weights_boot = weights[idx]
        treated_boot = T_boot == 1
        control_boot = T_boot == 0
        mu1_boot = np.mean(Y_boot[treated_boot])
        mu0_boot = np.average(Y_boot[control_boot], weights=weights_boot[control_boot])
        boot_atts.append(mu1_boot - mu0_boot)

    se = np.std(boot_atts)
    ci = (att - 1.96 * se, att + 1.96 * se)

    df['weight'] = weights
    return att, se, ci, ps

# Step 4: AIPTW (for Matched Data)
def estimate_aiptw(df, outcome, confounders):
    T = df['treatment'].values
    Y = df[outcome].values
    X = df[confounders].values if confounders else np.ones((len(df), 1))

    ps_model = LogisticRegression(max_iter=1000).fit(X, T)
    ps = ps_model.predict_proba(X)[:, 1]

    treated_idx = T == 1
    control_idx = T == 0

    outcome_model_treated = LinearRegression().fit(X[treated_idx], Y[treated_idx])
    outcome_model_control = LinearRegression().fit(X[control_idx], Y[control_idx])

    mu1_hat = outcome_model_treated.predict(X)
    mu0_hat = outcome_model_control.predict(X)

    term1 = (T * Y / ps) + (1 - T / ps) * mu1_hat
    term0 = ((1 - T) * Y / (1 - ps)) + (1 - (1 - T) / (1 - ps)) * mu0_hat

    mu1 = np.mean(term1)
    mu0 = np.mean(term0)
    ate = mu1 - mu0

    n = len(Y)
    boot_ates = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        T_boot = T[idx]
        Y_boot = Y[idx]
        X_boot = X[idx]
        ps_boot = ps_model.predict_proba(X_boot)[:, 1]
        mu1_boot = outcome_model_treated.predict(X_boot)
        mu0_boot = outcome_model_control.predict(X_boot)
        term1_boot = (T_boot * Y_boot / ps_boot) + (1 - T_boot / ps_boot) * mu1_boot
        term0_boot = ((1 - T_boot) * Y_boot / (1 - ps_boot)) + (1 - (1 - T_boot) / (1 - ps_boot)) * mu0_boot
        boot_ates.append(np.mean(term1_boot) - np.mean(term0_boot))

    se = np.std(boot_ates)
    ci = (ate - 1.96 * se, ate + 1.96 * se)

    return ate, se, ci, ps

# Step 5: Matching for ATT
def matching_att(df, outcome, ps):
    treated_idx = df[df['treatment'] == 1].index
    control_idx = df[df['treatment'] == 0].index

    treated_ps = ps[treated_idx]
    control_ps = ps[control_idx]

    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control_ps.reshape(-1, 1))
    distances, indices = nn.kneighbors(treated_ps.reshape(-1, 1))

    matched_control_idx = control_idx[indices.flatten()]
    matched_df = df.loc[list(treated_idx) + list(matched_control_idx)]

    att = matched_df[matched_df['treatment'] == 1][outcome].mean() - matched_df[matched_df['treatment'] == 0][outcome].mean()

    boot_atts = []
    for _ in range(200):
        idx = np.random.choice(len(matched_df), len(matched_df), replace=True)
        boot_df = matched_df.iloc[idx]
        att_boot = boot_df[boot_df['treatment'] == 1][outcome].mean() - boot_df[boot_df['treatment'] == 0][outcome].mean()
        boot_atts.append(att_boot)

    se = np.std(boot_atts)
    ci = (att - 1.96 * se, att + 1.96 * se)

    return att, se, ci

# Step 6: Stratified Analysis by Pet Type
def stratified_analysis(df, outcome, confounders):
    print("O1 value counts:")
    print(df['O1'].value_counts(dropna=False))

    pet_types = ['Dog', 'Cat']
    results = {}

    for pet_type in pet_types:
        print(f"\nStratified analysis for {pet_type} owners:")
        sub_df = df[df['O1'].str.contains(pet_type, case=False, na=False)]
        if len(sub_df) < 10 or sub_df['treatment'].nunique() < 2:
            print(f"Skipping {pet_type}: insufficient data (n={len(sub_df)})")
            continue

        att, se, ci, ps = estimate_iptw_att(sub_df, outcome, confounders)
        results[pet_type] = {'ATT': att, 'SE': se, 'CI': ci}
        print(f"{pet_type} ATT = {att:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")

    if not results:
        print("\nTrying binary O1 (pet owners vs. non-owners)...")
        pet_owners = df[df['O1'].str.contains('Yes', case=False, na=False)]
        if len(pet_owners) >= 10 and pet_owners['treatment'].nunique() == 2:
            att, se, ci, ps = estimate_iptw_att(pet_owners, outcome, confounders)
            results['Pet Owners'] = {'ATT': att, 'SE': se, 'CI': ci}
            print(f"Pet Owners ATT = {att:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")

    return results

# Step 7: Sensitivity Analysis (E-value and Austin Plot with Bias Curve)
def compute_e_value(ate, se):
    point_est = abs(ate)
    rr_point = np.exp(0.91 * point_est)
    e_value_point = rr_point + np.sqrt(rr_point * (rr_point - 1))

    lower_bound = abs(ate - 1.96 * se)
    if lower_bound > 0:
        rr_lower = np.exp(0.91 * lower_bound)
        e_value_lower = rr_lower + np.sqrt(rr_lower * (rr_lower - 1))
    else:
        e_value_lower = 1.0

    return e_value_point, e_value_lower

# Calculate Partial R^2 for Observed Confounders
def calculate_partial_r2(df, confounder_cols, target):
    r2_values = []
    for col in confounder_cols:
        X = df[[col]].dropna()
        y = df.loc[X.index, target]
        if len(X) == 0 or y.isna().all():
            continue
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        r2 = r2_score(y, y_pred)
        r2_values.append((col, r2))
    return r2_values

# Austin Plot with Bias Curve Styled Like the Provided Code
def austin_plot_sensitivity(df, outcome, confounder_cols, att_orig, se_orig, output_dir='plots', prefix=''):
    print(f"\nGenerating Austin plot with bias curve for sensitivity analysis ({prefix})...")

    T = df['treatment'].values
    Y = df[outcome].values

    # Calculate partial R^2 for observed confounders
    r2_t = calculate_partial_r2(df, confounder_cols, 'treatment')
    r2_y = calculate_partial_r2(df, confounder_cols, outcome)

    # Create a dictionary mapping confounder to (R^2_UT, R^2_UY)
    confounder_r2 = {}
    for (conf_t, r2_ut), (conf_y, r2_uy) in zip(r2_t, r2_y):
        if conf_t == conf_y:
            confounder_r2[conf_t] = (r2_ut, r2_uy)

    print("\nPartial R^2 of observed confounders:")
    for conf, (r2_ut, r2_uy) in confounder_r2.items():
        print(f"{conf}: R^2_UT = {r2_ut:.3f}, R^2_UY = {r2_uy:.3f}")

    # Generate distinct colors for each confounder
    colors = list(mcolors.TABLEAU_COLORS.values())[:len(confounder_r2)]
    if len(colors) < len(confounder_r2):
        colors.extend(list(mcolors.CSS4_COLORS.values())[:len(confounder_r2) - len(colors)])

    # Simulate unmeasured confounder U
    df['U'] = np.random.normal(0, 1, len(df))

    # Vary R^2 values for unmeasured confounding
    r2_values = np.linspace(0.0, 0.5, 21)  # 0.0 to 0.5 in 21 steps for contour plot
    r2_diagonal = np.linspace(0.0, 0.5, 50)  # Finer grid for the bias curve
    adjusted_atts = np.zeros((len(r2_values), len(r2_values)))

    sd_y = np.std(df[outcome])
    sd_t = np.std(df['treatment'])

    # Compute adjusted ATT for the contour plot
    for i, r2_ut in enumerate(r2_values):
        for j, r2_uy in enumerate(r2_values):
            df['T_adj'] = np.sqrt(1 - r2_ut) * df['treatment'] + np.sqrt(r2_ut) * df['U'] * sd_t
            df['Y_adj'] = df[outcome] + np.sqrt(r2_uy) * df['U'] * sd_y
            weights = np.where(df['T_adj'] > 0.5, 1, 0.5 / (1 - 0.5))
            treated_idx = df['T_adj'] > 0.5
            control_idx = df['T_adj'] <= 0.5
            mu1 = np.mean(df['Y_adj'][treated_idx])
            mu0 = np.average(df['Y_adj'][control_idx], weights=weights[control_idx])
            adjusted_atts[i, j] = mu1 - mu0

    # Compute ATT along the diagonal for the bias curve (R^2_UT = R^2_UY)
    diagonal_atts = []
    for r2 in r2_diagonal:
        df['T_adj'] = np.sqrt(1 - r2) * df['treatment'] + np.sqrt(r2) * df['U'] * sd_t
        df['Y_adj'] = df[outcome] + np.sqrt(r2) * df['U'] * sd_y
        weights = np.where(df['T_adj'] > 0.5, 1, 0.5 / (1 - 0.5))
        treated_idx = df['T_adj'] > 0.5
        control_idx = df['T_adj'] <= 0.5
        mu1 = np.mean(df['Y_adj'][treated_idx])
        mu0 = np.average(df['Y_adj'][control_idx], weights=weights[control_idx])
        diagonal_atts.append(mu1 - mu0)

    # Compute bias grid (Adjusted ATT - Original ATT)
    bias_grid = adjusted_atts - att_orig

    # Generate Austin Plot
    plt.figure(figsize=(12, 10))

    # Contour plot of adjusted ATT values
    contour = plt.contourf(r2_values, r2_values, adjusted_atts, levels=20, cmap='viridis')
    plt.colorbar(contour, label=f'Adjusted ATT', pad=0.1, fraction=0.046)

    # Contour for ATT = 0
    cs = plt.contour(r2_values, r2_values, adjusted_atts, levels=[0], colors='red', linestyles='dashed', linewidths=2)
    plt.clabel(cs, inline=True, fontsize=10, fmt='ATT = 0', colors='red')

    # Contours for 95% CI bounds, highlight lower CI as bias curve
    ci_lower = att_orig - 1.96 * se_orig
    ci_upper = att_orig + 1.96 * se_orig
    cs_ci = plt.contour(r2_values, r2_values, adjusted_atts, levels=[ci_lower, ci_upper], colors=['cyan', 'white'], linestyles=['solid', 'dotted'], linewidths=[2, 2])
    plt.clabel(cs_ci, inline=True, fontsize=10, fmt={ci_lower: f'Bias = Lower CI ({ci_lower:.3f})', ci_upper: f'CI Upper ({ci_upper:.3f})'}, colors=['cyan', 'white'])

    # Contour for bias = -1.96 * SE (should match the lower CI contour)
    bias_lower_ci = -1.96 * se_orig
    cs_bias = plt.contour(r2_values, r2_values, bias_grid, levels=[bias_lower_ci], colors='cyan', linestyles='solid', linewidths=2)
    plt.clabel(cs_bias, inline=True, fontsize=10, fmt=f'Bias = {bias_lower_ci:.3f}', colors='cyan')

    # Plot bias curve along the diagonal (R^2_UT = R^2_UY)
    plt.plot(r2_diagonal, r2_diagonal, linestyle='--', color='black', linewidth=2, label='R²_UT = R²_UY Path')

    # Plot observed confounders as scatter points
    markers = ['o', 's', '^', 'D', 'v', '<', '>', 'p', 'h', '*']
    for idx, (conf, (r2_ut, r2_uy)) in enumerate(confounder_r2.items()):
        plt.scatter(r2_ut, r2_uy, label=conf, color=colors[idx % len(colors)],
                    marker=markers[idx % len(markers)], s=150, alpha=0.8, edgecolors='black')

    # Add annotations
    plt.text(0.02, 0.48, f'Original ATT: {att_orig:.3f}', color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.5))
    plt.text(0.02, 0.44, f'CI: ({ci_lower:.3f}, {ci_upper:.3f})', color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.5))

    # Set axis labels and title
    plt.xlabel('Influence on Treatment (R²_UT)', fontsize=14, fontweight='bold')
    plt.ylabel('Influence on Outcome (R²_UY)', fontsize=14, fontweight='bold')
    plt.title(f'Austin Plot for Sensitivity to Unmeasured Confounding\n(Points Represent Observed Confounders)',
              fontsize=16, fontweight='bold', pad=20)

    # Customize legend
    num_confounders = len(confounder_r2) + 1  # +1 for the diagonal path
    num_cols = min(3, max(1, num_confounders // 4 + 1))
    plt.legend(title="Observed Confounders", fontsize=10, title_fontsize=12,
               loc='upper center', bbox_to_anchor=(0.5, -0.15),
               ncol=num_cols, frameon=True, edgecolor='black', markerscale=1.5)

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.25)
    plt.savefig(f'{output_dir}/austen_plot_with_bias_{prefix}.png', bbox_inches='tight')
    plt.close()

# Step 8: Diagnostics with SMD
def compute_smd(df, confounders, weights):
    treated = df[df['treatment'] == 1]
    control = df[df['treatment'] == 0]
    weights_t = weights[df['treatment'] == 1]
    weights_c = weights[df['treatment'] == 0]
    smd_results = {}

    for col in confounders:
        mean_t = np.average(treated[col], weights=weights_t)
        mean_c = np.average(control[col], weights=weights_c)
        var_t = np.average((treated[col] - mean_t)**2, weights=weights_t)
        var_c = np.average((control[col] - mean_c)**2, weights=weights_c)
        smd = (mean_t - mean_c) / np.sqrt((var_t + var_c) / 2)
        smd_results[col] = smd

    return smd_results

def plot_diagnostics(ps, df, output_dir='plots', prefix=''):
    os.makedirs(output_dir, exist_ok=True)

    plt.figure(figsize=(8, 6))
    sns.histplot(ps[df['treatment'] == 1], label='Treated', color='blue', alpha=0.5)
    sns.histplot(ps[df['treatment'] == 0], label='Control', color='orange', alpha=0.5)
    plt.title(f'Propensity Score Distribution ({prefix})')
    plt.xlabel('Propensity Score')
    plt.legend()
    plt.savefig(f'{output_dir}/{prefix}_ps_overlap.png')
    plt.close()

    plt.figure(figsize=(8, 6))
    sns.histplot(df['weight'], color='green', alpha=0.5)
    plt.title(f'IPTW Weight Distribution ({prefix})')
    plt.xlabel('Weight')
    plt.savefig(f'{output_dir}/{prefix}_weight_distribution.png')
    plt.close()

# Main Function
def main(data, o8b_treated='New', o8b_control='No'):
    df, outcome, confounders = preprocess_data(data, o8b_treated, o8b_control)

    print("\nEstimating ATT with IPTW...")
    att_iptw, se_iptw, ci_iptw, ps = estimate_iptw_att(df, outcome, confounders)
    print(f"IPTW ATT = {att_iptw:.3f}, SE = {se_iptw:.3f}, 95% CI = ({ci_iptw[0]:.3f}, {ci_iptw[1]:.3f})")

    print("\nPerforming propensity score matching...")
    matched_df, ps_matched = propensity_score_matching(df, confounders)
    ate_matched, se_matched, ci_matched, ps_matched_new = estimate_aiptw(matched_df, outcome, confounders)
    print(f"Matched ATE (AIPTW) = {ate_matched:.3f}, SE = {se_matched:.3f}, 95% CI = ({ci_matched[0]:.3f}, {ci_matched[1]:.3f})")

    att_matched, se_matched_att, ci_matched_att = matching_att(df, outcome, ps)
    print(f"Matched ATT = {att_matched:.3f}, SE = {se_matched_att:.3f}, 95% CI = ({ci_matched_att[0]:.3f}, {ci_matched_att[1]:.3f})")

    print("\nPerforming stratified analysis...")
    stratified_results = stratified_analysis(df, outcome, confounders)

    print("\nComputing E-value for IPTW ATT...")
    e_value_point, e_value_lower = compute_e_value(att_iptw, se_iptw)
    print(f"E-value (point estimate) = {e_value_point:.2f}")
    print(f"E-value (lower CI bound) = {e_value_lower:.2f}")

    # Generate Austin Plot with Bias Curve
    austin_plot_sensitivity(df, outcome, confounders, att_iptw, se_iptw, prefix='iptw_att')

    print("\nComputing SMD for IPTW ATT...")
    smd_results = compute_smd(df, confounders, df['weight'])
    for col, smd in smd_results.items():
        print(f"SMD for {col}: {smd:.3f}")

    plot_diagnostics(ps, df, prefix='full')
    plot_diagnostics(ps_matched_new, matched_df, prefix='matched')
    print("Diagnostic plots saved in 'plots' directory.")

# Run the analysis
if __name__ == "__main__":
    main(data, o8b_treated='New', o8b_control='No')

Starting preprocessing...
Initial shape: (1500, 23)
O8b value counts:
O8b
No      1259
Loss     173
New       68
Name: count, dtype: int64
Available columns:
['D2', 'region', 'O1', 'O8b', 'P1', 'D1', 'D5', 'D67', 'D4', 'D11', 'D8', 'E3', 'H2', 'H1Ax', 'H4', 'H56', 'S', 'QOL', 'GH', 'L1', 'H3', 'Q2', 'Q1_cat']
Treatment value counts:
treatment
0.0    1259
NaN     173
1.0      68
Name: count, dtype: int64
Shape after filtering treatment: (1327, 24)
Available confounders: ['D4', 'D5', 'D67', 'D11', 'H2', 'H1Ax', 'H56', 'P1', 'region']
Missing values before imputation:
QOL          0
treatment    0
D4           0
D5           0
D67          0
D11          0
H2           0
H1Ax         0
H56          0
P1           0
region       0
dtype: int64
Sample data for categorical confounders:
      D4           D5         D67    region   H2 H1Ax H56
0    Men  High school      Others  Atlantic   No   No  No
1    Men      College  Caucasians  Prairies   No   No  No
2    Men   University  Caucasians  

<ipython-input-11-4a44afc81b9d>:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = weights


Pet Owners ATT = -0.003, SE = 0.018, 95% CI = (-0.037, 0.032)

Computing E-value for IPTW ATT...
E-value (point estimate) = 1.12
E-value (lower CI bound) = 1.26

Generating Austin plot with bias curve for sensitivity analysis (iptw_att)...

Partial R^2 of observed confounders:
D4_Other or did not answer: R^2_UT = 0.002, R^2_UY = 0.000
D4_Women: R^2_UT = 0.001, R^2_UY = 0.004
D5_College: R^2_UT = 0.001, R^2_UY = 0.003
D5_High school: R^2_UT = 0.001, R^2_UY = 0.003
D5_University: R^2_UT = 0.003, R^2_UY = 0.016
D67_Others: R^2_UT = 0.002, R^2_UY = 0.002
region_British Columbia: R^2_UT = 0.003, R^2_UY = 0.000
region_Ontario: R^2_UT = 0.000, R^2_UY = 0.001
region_Prairies: R^2_UT = 0.001, R^2_UY = 0.001
region_Québec: R^2_UT = 0.002, R^2_UY = 0.006
H2_Yes: R^2_UT = 0.000, R^2_UY = 0.214
H1Ax_Yes: R^2_UT = 0.001, R^2_UY = 0.186
H56_Yes: R^2_UT = 0.002, R^2_UY = 0.000
D11: R^2_UT = 0.003, R^2_UY = 0.001
P1: R^2_UT = 0.011, R^2_UY = 0.021

Computing SMD for IPTW ATT...
SMD for D4_Other or did 

Estimate using 2SLS

In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(123)

# Step 1: Data Preprocessing
def preprocess_data(data):
    print("Starting preprocessing...")
    df = data.copy()

    print(f"Initial shape: {df.shape}")
    print("O1 value counts:")
    print(df['O1'].value_counts(dropna=False))

    print("Available columns:")
    print(df.columns.tolist())

    # Define treatment (O1)
    df['treatment'] = np.where(df['O1'] == 'Yes', 1,
                             np.where(df['O1'] == 'No', 0, np.nan))

    # Define instrument (P1) - keep as continuous
    df['instrument'] = df['P1']

    print("Treatment value counts:")
    print(df['treatment'].value_counts(dropna=False))

    print("Instrument (P1) summary:")
    print(df['instrument'].describe())

    # Filter to keep only valid treatment and instrument
    df = df[df['treatment'].notna() & df['instrument'].notna()]
    df = df.reset_index(drop=True)
    print(f"Shape after filtering: {df.shape}")

    if df.empty:
        raise ValueError("No rows remain after filtering.")

    outcome = 'QOL'
    if outcome not in df.columns:
        raise ValueError(f"Outcome column '{outcome}' not found in data.")

    confounders = ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
    confounders = [col for col in confounders if col in df.columns]
    print(f"Initial confounders: {confounders}")

    # Exclude confounders with all NaN values initially
    valid_confounders = []
    for col in confounders:
        if df[col].notna().any():
            valid_confounders.append(col)
        else:
            print(f"Excluding {col}: all values are NaN")
    confounders = valid_confounders
    print(f"Valid confounders: {confounders}")

    print("Missing values before imputation:")
    print(df[[outcome, 'treatment', 'instrument'] + confounders].isna().sum())

    print("Sample data for categorical confounders:")
    categorical_sample = [c for c in ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56'] if c in df.columns]
    if categorical_sample:
        print(df[categorical_sample].head())

    categorical_vars = ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
    categorical_vars = [var for var in categorical_vars if var in confounders]
    print(f"One-hot encoding categorical variables: {categorical_vars}")
    if categorical_vars:
        df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)

    confounder_cols = [col for col in df.columns if any(col.startswith(var + '_') for var in categorical_vars)]
    confounder_cols += [var for var in confounders if var not in categorical_vars]
    print(f"Confounder columns after encoding: {confounder_cols}")

    continuous_vars = ['D1', 'D8', 'D11']
    continuous_vars = [var for var in continuous_vars if var in df.columns]
    print(f"Continuous variables: {continuous_vars}")

    for var in continuous_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    valid_continuous_vars = [var for var in continuous_vars if df[var].notna().any()]
    print(f"Valid continuous variables (non-empty): {valid_continuous_vars}")

    if valid_continuous_vars:
        print(f"Imputing and standardizing: {valid_continuous_vars}")
        imputer = SimpleImputer(strategy='mean')
        df[valid_continuous_vars] = imputer.fit_transform(df[valid_continuous_vars])

        scaler = StandardScaler()
        df[valid_continuous_vars] = scaler.fit_transform(df[valid_continuous_vars])

    categorical_confounders = [col for col in confounder_cols if col not in continuous_vars]
    if categorical_confounders:
        missing_cat = df[categorical_confounders].isna().any().any()
        print(f"Missing values in categorical confounders: {missing_cat}")
        if missing_cat:
            print(f"Imputing categorical confounders: {categorical_confounders}")
            imputer_cat = SimpleImputer(strategy='most_frequent')
            df[categorical_confounders] = imputer_cat.fit_transform(df[categorical_confounders])

    if df[outcome].isna().any():
        print("Imputing missing QOL values...")
        imputer_outcome = SimpleImputer(strategy='mean')
        df[outcome] = imputer_outcome.fit_transform(df[[outcome]])

    # Drop columns that are entirely NaN after processing
    df = df.dropna(axis=1, how='all')
    print(f"Shape after dropping entirely NaN columns: {df.shape}")

    # Update confounder_cols to only include existing columns
    confounder_cols = [col for col in confounder_cols if col in df.columns]
    print(f"Updated confounder_cols: {confounder_cols}")

    print("Data types of confounder columns:")
    print(df[confounder_cols].dtypes)

    non_numeric_cols = df[confounder_cols].select_dtypes(exclude=['int64', 'float64', 'uint8', 'int32']).columns
    if non_numeric_cols.any():
        raise ValueError(f"Non-numeric confounders detected: {non_numeric_cols.tolist()}")

    print("Missing values after imputation:")
    final_cols = [outcome, 'treatment', 'instrument'] + confounder_cols
    print(df[final_cols].isna().sum())

    print(f"Final shape after preprocessing: {df.shape}")

    if df.empty:
        raise ValueError("No data remains after preprocessing.")

    return df, outcome, confounder_cols

# Step 2: Instrumental Variables (IV) for ATT
def iv_analysis_att(df, outcome, confounders):
    print("\nPerforming IV analysis for ATT...")
    # Check for sufficient variation in treatment and instrument
    print("Treatment variation check:")
    print(df['treatment'].value_counts(dropna=False))
    print("Instrument variation check:")
    print(df['instrument'].describe())

    if df['treatment'].nunique() < 2:
        raise ValueError("Insufficient variation in treatment variable.")
    if df['instrument'].nunique() < 2:
        raise ValueError("Insufficient variation in instrument variable.")

    # Filter confounders to only those with non-NaN values in df
    valid_confounders = [c for c in confounders if c in df.columns and df[c].notna().any()]
    print(f"Valid confounders for IV analysis: {valid_confounders}")

    # First stage: Regress treatment (O1) on instrument (P1) and confounders
    confounder_terms = [f'Q("{c}")' if ' ' in c else c for c in valid_confounders if not c.startswith('region_')]
    formula_first = 'treatment ~ instrument + ' + ' + '.join(confounder_terms)
    print(f"First-stage formula: {formula_first}")

    # Validate formula terms
    for term in confounder_terms:
        col = term.replace('Q("', '').replace('")', '')
        if col not in df.columns:
            raise ValueError(f"Column {col} not in DataFrame")

    first_stage = smf.ols(formula_first, data=df).fit()
    df['treatment_pred'] = first_stage.predict()

    # Check instrument strength (F-statistic on instrument)
    f_stat = first_stage.fvalue
    print(f"First-stage F-statistic: {f_stat:.2f}")
    if f_stat < 10:
        print("Warning: Weak instrument (F < 10). IV results may be unreliable.")

    # Second stage: Regress outcome on predicted treatment and confounders
    formula_second = f'{outcome} ~ treatment_pred + ' + ' + '.join(confounder_terms)
    print(f"Second-stage formula: {formula_second}")
    second_stage = smf.ols(formula_second, data=df).fit()

    att_iv = second_stage.params['treatment_pred']
    se_iv = second_stage.bse['treatment_pred']
    ci_iv = (att_iv - 1.96 * se_iv, att_iv + 1.96 * se_iv)

    print(f"IV ATT = {att_iv:.3f}, SE = {se_iv:.3f}, 95% CI = ({ci_iv[0]:.3f}, {ci_iv[1]:.3f})")
    return att_iv, se_iv, ci_iv

# Step 3: Instrumental Variables (IV) for ATE
def iv_analysis_ate(df, outcome, confounders):
    print("\nPerforming IV analysis for ATE...")
    # Check for sufficient variation in treatment and instrument
    print("Treatment variation check:")
    print(df['treatment'].value_counts(dropna=False))
    print("Instrument variation check:")
    print(df['instrument'].describe())

    if df['treatment'].nunique() < 2:
        raise ValueError("Insufficient variation in treatment variable.")
    if df['instrument'].nunique() < 2:
        raise ValueError("Insufficient variation in instrument variable.")

    # Filter confounders to only those with non-NaN values in df
    valid_confounders = [c for c in confounders if c in df.columns and df[c].notna().any()]
    print(f"Valid confounders for IV analysis: {valid_confounders}")

    # First stage: Regress treatment (O1) on instrument (P1) and confounders
    confounder_terms = [f'Q("{c}")' if ' ' in c else c for c in valid_confounders if not c.startswith('region_')]
    formula_first = 'treatment ~ instrument + ' + ' + '.join(confounder_terms)
    print(f"First-stage formula: {formula_first}")

    # Validate formula terms
    for term in confounder_terms:
        col = term.replace('Q("', '').replace('")', '')
        if col not in df.columns:
            raise ValueError(f"Column {col} not in DataFrame")

    first_stage = smf.ols(formula_first, data=df).fit()
    df['treatment_pred'] = first_stage.predict()

    # Check instrument strength (F-statistic on instrument)
    f_stat = first_stage.fvalue
    print(f"First-stage F-statistic: {f_stat:.2f}")
    if f_stat < 10:
        print("Warning: Weak instrument (F < 10). IV results may be unreliable.")

    # Second stage: Regress outcome on predicted treatment and confounders
    formula_second = f'{outcome} ~ treatment_pred + ' + ' + '.join(confounder_terms)
    print(f"Second-stage formula: {formula_second}")
    second_stage = smf.ols(formula_second, data=df).fit()

    ate_iv = second_stage.params['treatment_pred']
    se_iv = second_stage.bse['treatment_pred']
    ci_iv = (ate_iv - 1.96 * se_iv, ate_iv + 1.96 * se_iv)

    print(f"IV ATE = {ate_iv:.3f}, SE = {se_iv:.3f}, 95% CI = ({ci_iv[0]:.3f}, {ci_iv[1]:.3f})")
    return ate_iv, se_iv, ci_iv

# Step 4: Propensity Score Matching (PSM) for H1Ax=Yes Subgroup
def psm_analysis(df, outcome, confounders):
    print("\nPerforming PSM analysis for H1Ax=Yes subgroup...")
    # Subset to H1Ax=Yes
    sub_df = df[df['H1Ax_Yes'] == 1]
    sub_df = sub_df.dropna(axis=1, how='all')
    print(f"H1Ax=Yes subgroup size: {len(sub_df)}")

    if len(sub_df) < 10 or sub_df['treatment'].nunique() < 2:
        print("Skipping PSM: insufficient data or variation")
        return None, None, None

    # Filter confounders to only those with non-NaN values in sub_df
    valid_confounders = [c for c in confounders if c in sub_df.columns and sub_df[c].notna().any()]
    print(f"Valid confounders for PSM: {valid_confounders}")

    # Estimate propensity scores
    formula_ps = 'treatment ~ ' + ' + '.join([f'Q("{c}")' if ' ' in c else c for c in valid_confounders if not c.startswith('region_')])
    ps_model = smf.logit(formula_ps, data=sub_df).fit(disp=0)
    sub_df['ps'] = ps_model.predict()

    # PSM: Match treated to control units
    treated = sub_df[sub_df['treatment'] == 1]
    control = sub_df[sub_df['treatment'] == 0]

    nn = NearestNeighbors(n_neighbors=1, radius=0.25)
    nn.fit(control[['ps']])
    distances, indices = nn.kneighbors(treated[['ps']])

    valid_matches = distances.flatten() <= 0.25
    matched_treated = treated.index[valid_matches]
    matched_control = control.index[indices.flatten()[valid_matches]]

    if len(matched_treated) == 0:
        print("No matches within caliper. Skipping PSM.")
        return None, None, None

    matched_df = sub_df.loc[list(matched_treated) + list(matched_control)]
    print(f"Matched dataset size: {len(matched_df)}")

    # Compute ATT using matched sample
    att_psm = matched_df[matched_df['treatment'] == 1][outcome].mean() - matched_df[matched_df['treatment'] == 0][outcome].mean()

    # Bootstrap for SE
    boot_atts = []
    for _ in range(200):
        idx = np.random.choice(len(matched_df), len(matched_df), replace=True)
        boot_df = matched_df.iloc[idx]
        att_boot = boot_df[boot_df['treatment'] == 1][outcome].mean() - boot_df[boot_df['treatment'] == 0][outcome].mean()
        boot_atts.append(att_boot)

    se_psm = np.std(boot_atts)
    ci_psm = (att_psm - 1.96 * se_psm, att_psm + 1.96 * se_psm)

    print(f"H1Ax=Yes PSM ATT = {att_psm:.3f}, SE = {se_psm:.3f}, 95% CI = ({ci_psm[0]:.3f}, {ci_psm[1]:.3f})")
    return att_psm, se_psm, ci_psm

# Step 5: IV Analysis with H4 as Outcome
def iv_analysis_h4(df, confounders):
    print("\nPerforming IV analysis with H4 as outcome...")
    # Convert H4 to numeric: Much worse=-2, Somewhat worse=-1, About the same=0, Somewhat better=1, Much better=2
    df['H4_numeric'] = df['H4'].map({
        'Much worse': -2,
        'Somewhat worse': -1,
        'About the same': 0,
        'Somewhat better': 1,
        'Much better': 2
    })

    if df['H4_numeric'].isna().any():
        print("Warning: Missing or unmapped H4 values. Dropping missing...")
        df = df.dropna(subset=['H4_numeric'])

    print(f"Shape after dropping missing H4_numeric: {df.shape}")

    # Check for sufficient variation in treatment and instrument
    print("Treatment variation check:")
    print(df['treatment'].value_counts(dropna=False))
    print("Instrument variation check:")
    print(df['instrument'].describe())

    if df['treatment'].nunique() < 2:
        raise ValueError("Insufficient variation in treatment variable.")
    if df['instrument'].nunique() < 2:
        raise ValueError("Insufficient variation in instrument variable.")

    # Filter confounders to only those with non-NaN values in df
    valid_confounders = [c for c in confounders if c in df.columns and df[c].notna().any()]
    print(f"Valid confounders for IV analysis (H4): {valid_confounders}")

    # First stage: Regress treatment (O1) on instrument (P1) and confounders
    confounder_terms = [f'Q("{c}")' if ' ' in c else c for c in valid_confounders if not c.startswith('region_')]
    formula_first = 'treatment ~ instrument + ' + ' + '.join(confounder_terms)
    print(f"First-stage formula: {formula_first}")

    # Validate formula terms
    for term in confounder_terms:
        col = term.replace('Q("', '').replace('")', '')
        if col not in df.columns:
            raise ValueError(f"Column {col} not in DataFrame")

    first_stage = smf.ols(formula_first, data=df).fit()
    df['treatment_pred'] = first_stage.predict()

    # Check instrument strength (F-statistic on instrument)
    f_stat = first_stage.fvalue
    print(f"First-stage F-statistic: {f_stat:.2f}")
    if f_stat < 10:
        print("Warning: Weak instrument (F < 10). IV results may be unreliable.")

    # Second stage: Regress H4_numeric on predicted treatment and confounders
    formula_second = 'H4_numeric ~ treatment_pred + ' + ' + '.join(confounder_terms)
    print(f"Second-stage formula: {formula_second}")
    second_stage = smf.ols(formula_second, data=df).fit()

    att_iv = second_stage.params['treatment_pred']
    se_iv = second_stage.bse['treatment_pred']
    ci_iv = (att_iv - 1.96 * se_iv, att_iv + 1.96 * se_iv)

    print(f"IV ATT (H4_numeric) = {att_iv:.3f}, SE = {se_iv:.3f}, 95% CI = ({ci_iv[0]:.3f}, {ci_iv[1]:.3f})")
    return att_iv, se_iv, ci_iv

# Main Function
def main(data):
    # Preprocess data
    df, outcome, confounder_cols = preprocess_data(data)

    # Step 1: IV Analysis for ATT
    try:
        att_iv, se_iv, ci_iv = iv_analysis_att(df, outcome, confounder_cols)
    except Exception as e:
        print(f"Error in IV ATT analysis: {str(e)}")
        return

    # Step 2: IV Analysis for ATE
    try:
        ate_iv, se_ate, ci_ate = iv_analysis_ate(df, outcome, confounder_cols)
    except Exception as e:
        print(f"Error in IV ATE analysis: {str(e)}")
        return

    # Step 3: PSM for H1Ax=Yes Subgroup
    try:
        att_psm, se_psm, ci_psm = psm_analysis(df, outcome, confounder_cols)
    except Exception as e:
        print(f"Error in PSM analysis: {str(e)}")

    # Step 4: IV Analysis with H4 as Outcome
    try:
        att_h4, se_h4, ci_h4 = iv_analysis_h4(df, confounder_cols)
    except Exception as e:
        print(f"Error in IV analysis with H4: {str(e)}")
        return

    # Step 5: Explore Effect with H4 and L1
    explore_effect(df, confounder_cols)

# Explore Effect with H4 and L1 (already in script, moved to main for clarity)
def explore_effect(df, confounder_cols):
    print("\nExploring effect with H4 and L1...")
    print("H4 by treatment group:")
    print(df.groupby('treatment')['H4'].value_counts(normalize=True))

    print("L1 (loneliness) by treatment group:")
    print(df.groupby('treatment')['L1'].mean())

    # Stratify by H1Ax (pre-existing mental health condition)
    print("\nIV ATT by pre-existing mental health condition (H1Ax):")
    for condition in [1, 0]:  # H1Ax_Yes is binary after encoding
        sub_df = df[df['H1Ax_Yes'] == condition]
        sub_df = sub_df.dropna(axis=1, how='all')  # Drop entirely NaN columns in subgroup
        print(f"\nH1Ax={'Yes' if condition == 1 else 'No'} subgroup size: {len(sub_df)}")
        print(f"Treatment value counts in subgroup:")
        print(sub_df['treatment'].value_counts(dropna=False))
        print(f"Instrument summary in subgroup:")
        print(sub_df['instrument'].describe())

        if len(sub_df) < 10 or sub_df['treatment'].nunique() < 2 or sub_df['instrument'].nunique() < 2:
            print(f"Skipping H1Ax={'Yes' if condition == 1 else 'No'}: insufficient data or variation (n={len(sub_df)})")
            continue
        try:
            # Use only confounders present in sub_df
            sub_confounders = [c for c in confounder_cols if c in sub_df.columns]
            att, se, ci = iv_analysis_att(sub_df, 'QOL', sub_confounders)
            print(f"H1Ax={'Yes' if condition == 1 else 'No'} IV ATT = {att:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")
        except Exception as e:
            print(f"Error in H1Ax={'Yes' if condition == 1 else 'No'} subgroup: {str(e)}")

# Run the analysis
if __name__ == "__main__":
    main(data)

Starting preprocessing...
Initial shape: (1500, 23)
O1 value counts:
O1
No     750
Yes    750
Name: count, dtype: int64
Available columns:
['D2', 'region', 'O1', 'O8b', 'P1', 'D1', 'D5', 'D67', 'D4', 'D11', 'D8', 'E3', 'H2', 'H1Ax', 'H4', 'H56', 'S', 'QOL', 'GH', 'L1', 'H3', 'Q2', 'Q1_cat']
Treatment value counts:
treatment
0.0    750
1.0    750
Name: count, dtype: int64
Instrument (P1) summary:
count    1500.000000
mean       94.174000
std        21.055117
min        19.000000
25%        79.000000
50%        97.500000
75%       112.000000
max       126.000000
Name: instrument, dtype: float64
Shape after filtering: (1500, 25)
Initial confounders: ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
Valid confounders: ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
Missing values before imputation:
QOL           0
treatment     0
instrument    0
D1            0
D4            0
D5            0
D67           0
D8            0
D11           0
H2      

Estimate using LATE

In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(123)

# Step 1: Data Preprocessing
def preprocess_data(data):
    print("Starting preprocessing...")
    df = data.copy()

    print(f"Initial shape: {df.shape}")
    print("O1 value counts:")
    print(df['O1'].value_counts(dropna=False))

    print("Available columns:")
    print(df.columns.tolist())

    # Define treatment (O1)
    df['treatment'] = np.where(df['O1'] == 'Yes', 1,
                             np.where(df['O1'] == 'No', 0, np.nan))

    # Define instrument (P1) - keep as continuous initially
    df['instrument'] = df['P1']

    print("Treatment value counts:")
    print(df['treatment'].value_counts(dropna=False))

    print("Instrument (P1) summary:")
    print(df['instrument'].describe())

    # Filter to keep only valid treatment and instrument
    df = df[df['treatment'].notna() & df['instrument'].notna()]
    df = df.reset_index(drop=True)
    print(f"Shape after filtering: {df.shape}")

    if df.empty:
        raise ValueError("No rows remain after filtering.")

    outcome = 'QOL'
    if outcome not in df.columns:
        raise ValueError(f"Outcome column '{outcome}' not found in data.")

    confounders = ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
    confounders = [col for col in confounders if col in df.columns]
    print(f"Initial confounders: {confounders}")

    # Exclude confounders with all NaN values initially
    valid_confounders = []
    for col in confounders:
        if df[col].notna().any():
            valid_confounders.append(col)
        else:
            print(f"Excluding {col}: all values are NaN")
    confounders = valid_confounders
    print(f"Valid confounders: {confounders}")

    print("Missing values before imputation:")
    print(df[[outcome, 'treatment', 'instrument'] + confounders].isna().sum())

    print("Sample data for categorical confounders:")
    categorical_sample = [c for c in ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56'] if c in df.columns]
    if categorical_sample:
        print(df[categorical_sample].head())

    categorical_vars = ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
    categorical_vars = [var for var in categorical_vars if var in confounders]
    print(f"One-hot encoding categorical variables: {categorical_vars}")
    if categorical_vars:
        df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)

    confounder_cols = [col for col in df.columns if any(col.startswith(var + '_') for var in categorical_vars)]
    confounder_cols += [var for var in confounders if var not in categorical_vars]
    print(f"Confounder columns after encoding: {confounder_cols}")

    continuous_vars = ['D1', 'D8', 'D11']
    continuous_vars = [var for var in continuous_vars if var in df.columns]
    print(f"Continuous variables: {continuous_vars}")

    for var in continuous_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    valid_continuous_vars = [var for var in continuous_vars if df[var].notna().any()]
    print(f"Valid continuous variables (non-empty): {valid_continuous_vars}")

    if valid_continuous_vars:
        print(f"Imputing and standardizing: {valid_continuous_vars}")
        imputer = SimpleImputer(strategy='mean')
        df[valid_continuous_vars] = imputer.fit_transform(df[valid_continuous_vars])

        scaler = StandardScaler()
        df[valid_continuous_vars] = scaler.fit_transform(df[valid_continuous_vars])

    categorical_confounders = [col for col in confounder_cols if col not in continuous_vars]
    if categorical_confounders:
        missing_cat = df[categorical_confounders].isna().any().any()
        print(f"Missing values in categorical confounders: {missing_cat}")
        if missing_cat:
            print(f"Imputing categorical confounders: {categorical_confounders}")
            imputer_cat = SimpleImputer(strategy='most_frequent')
            df[categorical_confounders] = imputer_cat.fit_transform(df[categorical_confounders])

    if df[outcome].isna().any():
        print("Imputing missing QOL values...")
        imputer_outcome = SimpleImputer(strategy='mean')
        df[outcome] = imputer_outcome.fit_transform(df[[outcome]])

    # Drop columns that are entirely NaN after processing
    df = df.dropna(axis=1, how='all')
    print(f"Shape after dropping entirely NaN columns: {df.shape}")

    # Update confounder_cols to only include existing columns
    confounder_cols = [col for col in confounder_cols if col in df.columns]
    print(f"Updated confounder_cols: {confounder_cols}")

    print("Data types of confounder columns:")
    print(df[confounder_cols].dtypes)

    non_numeric_cols = df[confounder_cols].select_dtypes(exclude=['int64', 'float64', 'uint8', 'int32']).columns
    if non_numeric_cols.any():
        raise ValueError(f"Non-numeric confounders detected: {non_numeric_cols.tolist()}")

    print("Missing values after imputation:")
    final_cols = [outcome, 'treatment', 'instrument'] + confounder_cols
    print(df[final_cols].isna().sum())

    print(f"Final shape after preprocessing: {df.shape}")

    if df.empty:
        raise ValueError("No data remains after preprocessing.")

    return df, outcome, confounder_cols

# Step 2: LATE Estimation
def late_analysis(df, outcome, instrument_col, treatment_col, threshold=None):
    print(f"\nPerforming LATE analysis for outcome: {outcome}...")
    # Check for sufficient variation in treatment
    print("Treatment variation check:")
    print(df[treatment_col].value_counts(dropna=False))

    if df[treatment_col].nunique() < 2:
        raise ValueError("Insufficient variation in treatment variable.")

    # Dichotomize the instrument if continuous
    if threshold is None:
        threshold = df[instrument_col].median()
    print(f"Using threshold {threshold} to dichotomize instrument...")
    df['Z'] = (df[instrument_col] > threshold).astype(int)

    print("Instrument (Z) variation check:")
    print(df['Z'].value_counts(dropna=False))

    if df['Z'].nunique() < 2:
        raise ValueError("Insufficient variation in instrument variable after dichotomization.")

    # Estimate LATE using the Wald estimator
    # LATE = Cov(Z, Y) / Cov(Z, T)
    cov_zy = np.cov(df['Z'], df[outcome])[0, 1]
    cov_zt = np.cov(df['Z'], df[treatment_col])[0, 1]

    if cov_zt == 0:
        raise ValueError("Instrument does not affect treatment (Cov(Z, T) = 0).")

    late = cov_zy / cov_zt
    print(f"Cov(Z, Y) = {cov_zy:.4f}, Cov(Z, T) = {cov_zt:.4f}, LATE = {late:.4f}")

    # Bootstrap for standard error and CI
    n = len(df)
    boot_lates = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        boot_df = df.iloc[idx]
        boot_cov_zy = np.cov(boot_df['Z'], boot_df[outcome])[0, 1]
        boot_cov_zt = np.cov(boot_df['Z'], boot_df[treatment_col])[0, 1]
        if boot_cov_zt == 0:
            continue
        boot_late = boot_cov_zy / boot_cov_zt
        boot_lates.append(boot_late)

    se = np.std(boot_lates)
    ci = (late - 1.96 * se, late + 1.96 * se)

    print(f"LATE = {late:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")
    return late, se, ci

# Step 3: Explore Effect with H4 and L1
def explore_effect(df, confounder_cols):
    print("\nExploring effect with H4 and L1...")
    print("H4 by treatment group:")
    print(df.groupby('treatment')['H4'].value_counts(normalize=True))

    print("L1 (loneliness) by treatment group:")
    print(df.groupby('treatment')['L1'].mean())

    # Stratify by H1Ax (pre-existing mental health condition)
    print("\nLATE by pre-existing mental health condition (H1Ax):")
    for condition in [1, 0]:  # H1Ax_Yes is binary after encoding
        sub_df = df[df['H1Ax_Yes'] == condition]
        sub_df = sub_df.dropna(axis=1, how='all')  # Drop entirely NaN columns in subgroup
        print(f"\nH1Ax={'Yes' if condition == 1 else 'No'} subgroup size: {len(sub_df)}")
        print(f"Treatment value counts in subgroup:")
        print(sub_df['treatment'].value_counts(dropna=False))
        print(f"Instrument summary in subgroup:")
        print(sub_df['instrument'].describe())

        if len(sub_df) < 10 or sub_df['treatment'].nunique() < 2 or sub_df['instrument'].nunique() < 2:
            print(f"Skipping H1Ax={'Yes' if condition == 1 else 'No'}: insufficient data or variation (n={len(sub_df)})")
            continue
        try:
            late, se, ci = late_analysis(sub_df, 'QOL', 'instrument', 'treatment')
            print(f"H1Ax={'Yes' if condition == 1 else 'No'} LATE = {late:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")
        except Exception as e:
            print(f"Error in H1Ax={'Yes' if condition == 1 else 'No'} subgroup: {str(e)}")

# Main Function
def main(data):
    # Preprocess data
    df, outcome, confounder_cols = preprocess_data(data)

    # Step 1: LATE Analysis for QOL
    try:
        late_qol, se_qol, ci_qol = late_analysis(df, 'QOL', 'instrument', 'treatment')
    except Exception as e:
        print(f"Error in LATE analysis for QOL: {str(e)}")
        return

    # Step 2: LATE Analysis for H4_numeric
    # Convert H4 to numeric
    df['H4_numeric'] = df['H4'].map({
        'Much worse': -2,
        'Somewhat worse': -1,
        'About the same': 0,
        'Somewhat better': 1,
        'Much better': 2
    })

    if df['H4_numeric'].isna().any():
        print("Warning: Missing or unmapped H4 values. Dropping missing...")
        df = df.dropna(subset=['H4_numeric'])

    try:
        late_h4, se_h4, ci_h4 = late_analysis(df, 'H4_numeric', 'instrument', 'treatment')
    except Exception as e:
        print(f"Error in LATE analysis for H4_numeric: {str(e)}")
        return

    # Step 3: Explore Effect
    explore_effect(df, confounder_cols)

# Run the analysis
if __name__ == "__main__":
    main(data)

Starting preprocessing...
Initial shape: (1500, 23)
O1 value counts:
O1
No     750
Yes    750
Name: count, dtype: int64
Available columns:
['D2', 'region', 'O1', 'O8b', 'P1', 'D1', 'D5', 'D67', 'D4', 'D11', 'D8', 'E3', 'H2', 'H1Ax', 'H4', 'H56', 'S', 'QOL', 'GH', 'L1', 'H3', 'Q2', 'Q1_cat']
Treatment value counts:
treatment
0.0    750
1.0    750
Name: count, dtype: int64
Instrument (P1) summary:
count    1500.000000
mean       94.174000
std        21.055117
min        19.000000
25%        79.000000
50%        97.500000
75%       112.000000
max       126.000000
Name: instrument, dtype: float64
Shape after filtering: (1500, 25)
Initial confounders: ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
Valid confounders: ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
Missing values before imputation:
QOL           0
treatment     0
instrument    0
D1            0
D4            0
D5            0
D67           0
D8            0
D11           0
H2      

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.colors as mcolors

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Step 1: Data Preprocessing (Including Observed Confounders)
def preprocess_data(data):
    print("Starting preprocessing...")
    df = data.copy()

    print(f"Initial shape: {df.shape}")
    print("O1 value counts:")
    print(df['O1'].value_counts(dropna=False))

    print("Available columns:")
    print(df.columns.tolist())

    # Define treatment (O1)
    df['treatment'] = np.where(df['O1'] == 'Yes', 1,
                             np.where(df['O1'] == 'No', 0, np.nan))

    # Define instrument (P1)
    df['instrument'] = df['P1']

    print("Treatment value counts:")
    print(df['treatment'].value_counts(dropna=False))

    print("Instrument (P1) summary:")
    print(df['instrument'].describe())

    # Filter to keep only valid treatment and instrument
    df = df[df['treatment'].notna() & df['instrument'].notna()]
    df = df.reset_index(drop=True)
    print(f"Shape after filtering: {df.shape}")

    if df.empty:
        raise ValueError("No rows remain after filtering.")

    # Ensure QOL and H4 are present
    if 'QOL' not in df.columns:
        raise ValueError("Outcome column 'QOL' not found in data.")
    if 'H4' not in df.columns:
        raise ValueError("Outcome column 'H4' not found in data.")

    # Convert H4 to numeric
    df['H4_numeric'] = df['H4'].map({
        'Much worse': -2,
        'Somewhat worse': -1,
        'About the same': 0,
        'Somewhat better': 1,
        'Much better': 2
    })

    if df['H4_numeric'].isna().any():
        print("Warning: Missing or unmapped H4 values. Dropping missing...")
        df = df.dropna(subset=['H4_numeric'])

    # Handle confounders
    confounders = ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
    confounders = [col for col in confounders if col in df.columns]
    print(f"Initial confounders: {confounders}")

    # Exclude confounders with all NaN values
    valid_confounders = []
    for col in confounders:
        if df[col].notna().any():
            valid_confounders.append(col)
        else:
            print(f"Excluding {col}: all values are NaN")
    confounders = valid_confounders
    print(f"Valid confounders: {confounders}")

    # One-hot encode categorical confounders
    categorical_vars = ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
    categorical_vars = [var for var in categorical_vars if var in confounders]
    print(f"One-hot encoding categorical variables: {categorical_vars}")
    if categorical_vars:
        df = pd.get_dummies(df, columns=categorical_vars, drop_first=True, dtype=int)

    confounder_cols = [col for col in df.columns if any(col.startswith(var + '_') for var in categorical_vars)]
    continuous_vars = ['D1', 'D8', 'D11']
    continuous_vars = [var for var in continuous_vars if var in df.columns]
    confounder_cols += [var for var in continuous_vars if var in df.columns]
    print(f"Confounder columns after encoding: {confounder_cols}")

    for var in continuous_vars:
        if var in df.columns:
            df[var] = pd.to_numeric(df[var], errors='coerce')

    valid_continuous_vars = [var for var in continuous_vars if var in df.columns and df[var].notna().any()]
    print(f"Valid continuous variables (non-empty): {valid_continuous_vars}")

    if valid_continuous_vars:
        print(f"Imputing and standardizing: {valid_continuous_vars}")
        imputer = SimpleImputer(strategy='mean')
        df[valid_continuous_vars] = imputer.fit_transform(df[valid_continuous_vars])

        scaler = StandardScaler()
        df[valid_continuous_vars] = scaler.fit_transform(df[valid_continuous_vars])

    categorical_confounders = [col for col in confounder_cols if col not in continuous_vars]
    if categorical_confounders:
        missing_cat = df[categorical_confounders].isna().any().any()
        print(f"Missing values in categorical confounders: {missing_cat}")
        if missing_cat:
            print(f"Imputing categorical confounders: {categorical_confounders}")
            imputer_cat = SimpleImputer(strategy='most_frequent')
            df[categorical_confounders] = imputer_cat.fit_transform(df[categorical_confounders])

    print(f"Final shape after preprocessing: {df.shape}")
    return df, confounder_cols

# Step 2: LATE Estimation
def late_analysis(df, outcome, instrument_col, treatment_col, threshold=None):
    print(f"\nPerforming LATE analysis for outcome: {outcome}...")
    # Check for sufficient variation in treatment
    print("Treatment variation check:")
    print(df[treatment_col].value_counts(dropna=False))

    if df[treatment_col].nunique() < 2:
        raise ValueError("Insufficient variation in treatment variable.")

    # Dichotomize the instrument if continuous
    if threshold is None:
        threshold = df[instrument_col].median()
    print(f"Using threshold {threshold} to dichotomize instrument...")
    df['Z'] = (df[instrument_col] > threshold).astype(int)

    print("Instrument (Z) variation check:")
    print(df['Z'].value_counts(dropna=False))

    if df['Z'].nunique() < 2:
        raise ValueError("Insufficient variation in instrument variable after dichotomization.")

    # Estimate LATE using the Wald estimator
    cov_zy = np.cov(df['Z'], df[outcome])[0, 1]
    cov_zt = np.cov(df['Z'], df[treatment_col])[0, 1]

    if cov_zt == 0:
        raise ValueError("Instrument does not affect treatment (Cov(Z, T) = 0).")

    late = cov_zy / cov_zt
    print(f"Cov(Z, Y) = {cov_zy:.4f}, Cov(Z, T) = {cov_zt:.4f}, LATE = {late:.4f}")

    # Bootstrap for standard error and CI
    n = len(df)
    boot_lates = []
    for _ in range(200):
        idx = np.random.choice(n, n, replace=True)
        boot_df = df.iloc[idx]
        boot_cov_zy = np.cov(boot_df['Z'], boot_df[outcome])[0, 1]
        boot_cov_zt = np.cov(boot_df['Z'], boot_df[treatment_col])[0, 1]
        if boot_cov_zt == 0:
            continue
        boot_late = boot_cov_zy / boot_cov_zt
        boot_lates.append(boot_late)

    se = np.std(boot_lates)
    ci = (late - 1.96 * se, late + 1.96 * se)

    print(f"LATE = {late:.3f}, SE = {se:.3f}, 95% CI = ({ci[0]:.3f}, {ci[1]:.3f})")
    return late, se, ci, threshold

# Step 3: Calculate Partial R^2 for Observed Confounders
def calculate_partial_r2(df, confounder_cols, target):
    r2_values = []
    for col in confounder_cols:
        # Prepare data
        X = df[[col]].dropna()
        y = df.loc[X.index, target]
        if len(X) == 0 or y.isna().all():
            continue
        # Fit linear regression
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        # Calculate R^2
        r2 = r2_score(y, y_pred)
        r2_values.append((col, r2))
    return r2_values

# Step 4: Sensitivity Analysis and Austin Plot with Adjusted Legends
def sensitivity_analysis_austen_plot(df, outcome, instrument_col, treatment_col, confounder_cols, threshold, outcome_name, late_orig, se_orig):
    print(f"\nGenerating sensitivity analysis and Austin plot for outcome: {outcome} (Simulating unmeasured confounder U)...")

    # Dichotomize the instrument
    df['Z'] = (df[instrument_col] > threshold).astype(int)

    # Calculate partial R^2 for observed confounders
    r2_z = calculate_partial_r2(df, confounder_cols, 'Z')
    r2_y = calculate_partial_r2(df, confounder_cols, outcome)

    # Create a dictionary mapping confounder to (R^2_UZ, R^2_UY)
    confounder_r2 = {}
    for (conf_z, r2_uz), (conf_y, r2_uy) in zip(r2_z, r2_y):
        if conf_z == conf_y:
            confounder_r2[conf_z] = (r2_uz, r2_uy)

    print("\nPartial R^2 of observed confounders:")
    for conf, (r2_uz, r2_uy) in confounder_r2.items():
        print(f"{conf}: R^2_UZ = {r2_uz:.3f}, R^2_UY = {r2_uy:.3f}")

    # Generate distinct colors for each confounder
    colors = list(mcolors.TABLEAU_COLORS.values())[:len(confounder_r2)]
    if len(colors) < len(confounder_r2):
        colors.extend(list(mcolors.CSS4_COLORS.values())[:len(confounder_r2) - len(colors)])

    # Generate hypothetical unmeasured confounder U
    df['U'] = np.random.normal(0, 1, len(df))

    # Vary R^2 values for unmeasured confounding
    r2_values = np.linspace(0.0, 0.5, 21)  # 0.0 to 0.5 in 21 steps
    adjusted_lates = np.zeros((len(r2_values), len(r2_values)))

    # Standard deviation of the outcome for scaling
    sd_y = np.std(df[outcome])

    for i, r2_uz in enumerate(r2_values):
        for j, r2_uy in enumerate(r2_values):
            # Adjust Z and Y for unmeasured confounding
            df['Z_adj'] = np.sqrt(1 - r2_uz) * df['Z'] + np.sqrt(r2_uz) * df['U']
            df['Y_adj'] = df[outcome] + np.sqrt(r2_uy) * df['U'] * sd_y

            # Recalculate LATE
            cov_zy_adj = np.cov(df['Z_adj'], df['Y_adj'])[0, 1]
            cov_zt_adj = np.cov(df['Z_adj'], df[treatment_col])[0, 1]
            if cov_zt_adj == 0:
                adjusted_lates[i, j] = np.nan
            else:
                adjusted_lates[i, j] = cov_zy_adj / cov_zt_adj

    # Generate Austin Plot with Adjusted Legends
    plt.figure(figsize=(12, 10))

    # Contour plot of adjusted LATE values
    contour = plt.contourf(r2_values, r2_values, adjusted_lates, levels=20, cmap='viridis')
    plt.colorbar(contour, label=f'Adjusted LATE for {outcome_name}', pad=0.1, fraction=0.046)

    # Contour for LATE = 0 (null effect)
    plt.contour(r2_values, r2_values, adjusted_lates, levels=[0], colors='red', linestyles='dashed', linewidths=2)

    # Contours for significance bounds (95% CI)
    ci_lower = late_orig - 1.96 * se_orig
    ci_upper = late_orig + 1.96 * se_orig
    plt.contour(r2_values, r2_values, adjusted_lates, levels=[ci_lower, ci_upper], colors='white', linestyles='dotted', linewidths=2)

    # Plot observed confounders as scatter points with distinct colors and markers
    markers = ['o', 's', '^', 'D', 'v', '<', '>', 'p', 'h', '*']  # Different markers for each confounder
    for idx, (conf, (r2_uz, r2_uy)) in enumerate(confounder_r2.items()):
        plt.scatter(r2_uz, r2_uy, label=conf, color=colors[idx % len(colors)],
                    marker=markers[idx % len(markers)], s=150, alpha=0.8, edgecolors='black')

    # Add annotations with larger, bold font
    plt.text(0.02, 0.48, f'Original LATE: {late_orig:.3f}', color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.5))
    plt.text(0.02, 0.44, f'CI: ({ci_lower:.3f}, {ci_upper:.3f})', color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.5))

    # Set axis labels and title with larger, bold font
    plt.xlabel('$R^2_{UZ}$ (Association with Instrument)', fontsize=14, fontweight='bold')
    plt.ylabel('$R^2_{UY}$ (Association with Outcome)', fontsize=14, fontweight='bold')
    plt.title(f'Austin Plot for {outcome_name}: Sensitivity to Unmeasured Confounding\n(Points Represent Observed Confounders)',
              fontsize=16, fontweight='bold', pad=20)

    # Customize legend to prevent overlap
    # Position legend below the plot with multiple columns if needed
    num_confounders = len(confounder_r2)
    num_cols = min(3, max(1, num_confounders // 4 + 1))  # Adjust number of columns based on number of confounders
    plt.legend(title="Observed Confounders", fontsize=10, title_fontsize=12,
               loc='upper center', bbox_to_anchor=(0.5, -0.15),
               ncol=num_cols, frameon=True, edgecolor='black', markerscale=1.5)

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.25)  # Add extra space at the bottom for the legend
    plt.savefig(f'austen_plot_{outcome_name.lower()}_with_observed_confounders.png', bbox_inches='tight')
    plt.close()

    return adjusted_lates

# Main Function
def main(data):
    # Preprocess data
    df, confounder_cols = preprocess_data(data)

    # Step 1: LATE Analysis for QOL
    try:
        late_qol, se_qol, ci_qol, threshold_qol = late_analysis(df, 'QOL', 'instrument', 'treatment')
    except Exception as e:
        print(f"Error in LATE analysis for QOL: {str(e)}")
        return

    # Step 2: LATE Analysis for H4_numeric
    try:
        late_h4, se_h4, ci_h4, threshold_h4 = late_analysis(df, 'H4_numeric', 'instrument', 'treatment')
    except Exception as e:
        print(f"Error in LATE analysis for H4_numeric: {str(e)}")
        return

    # Step 3: Generate Austin Plots with Observed Confounders
    try:
        # For QOL
        adjusted_lates_qol = sensitivity_analysis_austen_plot(df, 'QOL', 'instrument', 'treatment', confounder_cols, threshold_qol, 'QOL', late_qol, se_qol)
        # For H4_numeric
        adjusted_lates_h4 = sensitivity_analysis_austen_plot(df, 'H4_numeric', 'instrument', 'treatment', confounder_cols, threshold_h4, 'H4_numeric', late_h4, se_h4)
    except Exception as e:
        print(f"Error in generating Austin plots: {str(e)}")

# Run the analysis
if __name__ == "__main__":
    # Assuming 'data' is your DataFrame
    # Replace 'data' with your actual DataFrame variable
    main(data)

Starting preprocessing...
Initial shape: (1500, 23)
O1 value counts:
O1
No     750
Yes    750
Name: count, dtype: int64
Available columns:
['D2', 'region', 'O1', 'O8b', 'P1', 'D1', 'D5', 'D67', 'D4', 'D11', 'D8', 'E3', 'H2', 'H1Ax', 'H4', 'H56', 'S', 'QOL', 'GH', 'L1', 'H3', 'Q2', 'Q1_cat']
Treatment value counts:
treatment
0.0    750
1.0    750
Name: count, dtype: int64
Instrument (P1) summary:
count    1500.000000
mean       94.174000
std        21.055117
min        19.000000
25%        79.000000
50%        97.500000
75%       112.000000
max       126.000000
Name: instrument, dtype: float64
Shape after filtering: (1500, 25)
Initial confounders: ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
Valid confounders: ['D1', 'D4', 'D5', 'D67', 'D8', 'D11', 'H2', 'H1Ax', 'H56', 'region']
One-hot encoding categorical variables: ['D4', 'D5', 'D67', 'region', 'H2', 'H1Ax', 'H56']
Confounder columns after encoding: ['D4_Other or did not answer', 'D4_Women', 'D5_College', 'D5